# Banderas rojas 

Este notebook contiene el código que importa los datos de base de datos SQL hasta la generación de salida con detalle de alertas para cada convocatoria. 

1. Configuración: Librerias, directorios y configuraciones de Jupyter notebook
2. Data: Importación de datos desde base de datos SQL
3. Preprocesamiento de los datos
4. Banderas: Codigo de generación de cada una de las banderas
5. Salidas: 
    - Postprocesamiento que vincula el dataframe que contiene convocatorias 2021 y sus caracteristicas con cada una de las banderas generadas en el paso previo.
    - Generacion de Excels o CSV con salidas del proceso: Tiempos de procesamiento de cada bandera, cantidad de convocatorias que tienen alerta por cada bandera, cantidad de convocatorias con alertas por entidad, diccionario de grupos de items y malla que contiene los datos desagregados de caracteristicas y banderas a nivel convocatoria


## 1. Configuración

In [1]:
# Librerias
import pandas as pd
from pathlib import Path
import os
from sqlalchemy import create_engine
import numpy as np
import datetime
from datetime import datetime
import math
import datetime
import re

In [2]:
# Display options
pd.options.display.max_colwidth = 200
pd.options.display.float_format = '{:,.0f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)

In [3]:
#  Working directory
try:
    path = Path(__file__).parent # para correr en consola
except NameError:
    path = Path('/Users/mac/Documents/IDB') #para correr en spyder
os.chdir(path)

In [4]:
# Credenciales conexion nueva BD
import etl.etapa_1_nueva_base.credentials_newdb as credentials

In [5]:
# Funcion para registrar el tiempo que tarda en hacer cada bandera

tiempos = pd.DataFrame(columns = ['numero','tipo','tiempo_procesamiento'])

def registro_tiempo_procesamiento(nro_bandera,start,end, tiempos):
    '''
    Agrega a dataframe de control de tiempos de procesamiento el tiempo que tarda en computarse cada bandera
    
    Params
    ------
    nro_bandera (int): Número de bandera que se procesa
    start (datetime): Fecha y hora de inicio de procesamiento de bandera
    end (datetime): Fecha y hora de fin de procesamiento de bandera
    df_tiempos (dataframe) : Dataframe donde se van registrando los tiempos de procesamiento de cada bandera
    
    Returns
    ------
    
    df_tiempos (dataframe): Dataframe que contiene el computo del tiempo transcurrido entre incio y fin de proceso 
    de la bandera agregada
    '''
    # Calculo tiempo de proceso
    tiempo_proceso = end - start
    
    # Agrego a DF
    tiempos = tiempos.append({'numero':nro_bandera,
                              'tipo':'Bandera',
                             'tiempo_procesamiento': tiempo_proceso}, ignore_index = True)
    return(tiempos)

## 2. Data

In [6]:
# Conexion a BD
hostname = credentials.hostname
username = credentials.username
password = credentials.password
database = credentials.database
port = credentials.port

# Create an engine instance
alchemyEngine = create_engine(f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database}', pool_recycle=3600);

In [7]:
# Connect to PostgreSQL server
dbConnection    = alchemyEngine.connect();

In [8]:
# Base convocatorias
convocatorias = pd.read_sql_table('F_CONVOCATORIAS', con = dbConnection) 

In [9]:
# Base convocatorias a nivel convocatoria
convocatorias_agg = pd.read_sql_table('f_convocatorias_ccunico', con = dbConnection) 

In [10]:
# Postores
postores = pd.read_sql_table('F_POSTOR', con = dbConnection) 

In [11]:
# Postores a nivel convocatoria
postores_agg = pd.read_sql_table('f_postor_ccunico', con = dbConnection) 

In [12]:
# Participantes
participantes = pd.read_sql_table('F_PROVEEDOR_PARTICIPANTE', con = dbConnection) 

In [13]:
# Participantes a nivel convocatoria
participantes_agg = pd.read_sql_table('f_participantes_ccunico', con = dbConnection) 

In [14]:
# Base adjudicaciones 
adjudicaciones = pd.read_sql_table('F_ADJUDICACIONES', con = dbConnection) 

In [15]:
# Base adjudicaciones a nivel convocatoria
adjudicaciones_agg = pd.read_sql_table('f_adjudicaciones_ccunico', con = dbConnection) 

In [16]:
# Base contratos a nivel convocatoria
contratos = pd.read_sql_table('F_CONTRATOS_NEW', con = dbConnection) 

In [17]:
# Base contratos a nivel convocatoria
contratos_agg = pd.read_sql_table('f_contratos_ccunico', con = dbConnection) 

In [18]:
# PAC
pac = pd.read_sql_table('F_PAC', con = dbConnection) 

In [19]:
# PAC a nivel anio,ruc_entidad,n referencia
pac_agg = pd.read_sql_table('f_pac_ccunico', con = dbConnection) 

In [20]:
# SUNAT
sunat = pd.read_excel('data/sunat/SUNAT.xlsx')

In [21]:
# Tablas para hacer match entre convocatorias y PAC
match2018 = pd.read_excel('data/match_convocatorias_pac/Convocatoria 2018.xlsx')
match2019 = pd.read_excel('data/match_convocatorias_pac/Convocatoria 2019.xlsx')
match2020 = pd.read_excel('data/match_convocatorias_pac/Convocatoria 2020.xlsx')
match2021 = pd.read_excel('data/match_convocatorias_pac/Convocatoria 2021.xlsx')

In [22]:
match = match2018.append([match2019, match2020,match2021])

In [23]:
# Traigo convocatorias de la base de datos abiertos para completar los datos que faltan en la nueva base
# Conexion a BD
hostname = credentials.hostname_old
username = credentials.username_old
password = credentials.password_old
database = credentials.database_old
port = credentials.port_old

# Create an engine instance
alchemyEngine = create_engine(f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database}', pool_recycle=3600);

dbConnection  = alchemyEngine.connect();

convocatorias_old_agg = pd.read_sql_table('convocatorias_ccunico_soles', con = dbConnection) 

In [24]:
# Feriados para calculo de fechas solo dias habiles
feriados = pd.read_excel('data/nueva_base/FeriadosPeru.xlsx')
fecha_feriados=np.array(feriados['fecha'],dtype='datetime64[D]')

In [25]:
# Clasificacion de Entes para Bandera 1
entes_tipoa= pd.read_excel('data/data_complementaria/Bandera1_Clasicación de Entidades.xlsx', sheet_name = 'TIPO A', skiprows = 1)
entes_tipob= pd.read_excel('data/data_complementaria/Bandera1_Clasicación de Entidades.xlsx', sheet_name = 'TIPO B', skiprows = 1)
entes_tipoc= pd.read_excel('data/data_complementaria/Bandera1_Clasicación de Entidades.xlsx', sheet_name = 'TIPO C', skiprows = 1)
entes_tipod= pd.read_excel('data/data_complementaria/Bandera1_Clasicación de Entidades.xlsx', sheet_name = 'TIPO D', skiprows = 1)
entes_tipoe= pd.read_excel('data/data_complementaria/Bandera1_Clasicación de Entidades.xlsx', sheet_name = 'TIPO E', skiprows = 1)

In [26]:
# DGR 2021
dgr_aso = pd.read_excel('data/data_complementaria/Bandera_8_Data DGR 2021.xlsx', sheet_name = 'ASO2021')
dgr_dict = pd.read_excel('data/data_complementaria/Bandera_8_Data DGR 2021.xlsx', sheet_name = 'DICT2021')

## 3. Preprocesamiento

In [27]:
def convertir_a_minuscula(df):
    df.columns = map(str.lower, df.columns)
    return

def agregar_anio(df,variable):
    df['anio'] = df[variable].dt.year

In [28]:
# Columns to lowercase
convertir_a_minuscula(convocatorias)
convertir_a_minuscula(adjudicaciones)
convertir_a_minuscula(contratos)
convertir_a_minuscula(postores)
convertir_a_minuscula(participantes)
convertir_a_minuscula(pac)
convertir_a_minuscula(match)
convertir_a_minuscula(sunat)
convertir_a_minuscula(dgr_aso)
convertir_a_minuscula(dgr_dict)

convertir_a_minuscula(convocatorias_agg)
convertir_a_minuscula(adjudicaciones_agg)
convertir_a_minuscula(contratos_agg)
convertir_a_minuscula(pac_agg)
convertir_a_minuscula(postores_agg)
convertir_a_minuscula(participantes_agg)

# Agrego año de convocatoria
agregar_anio(convocatorias,'fecha_convocatoria')
agregar_anio(convocatorias_agg,'fecha_convocatoria')
agregar_anio(adjudicaciones,'fechaconvocatoria')
agregar_anio(adjudicaciones_agg,'fechaconvocatoria')
agregar_anio(contratos,'fechaconvocatoria')
agregar_anio(contratos_agg,'fechaconvocatoria')


In [29]:
# Convierto a Int todos los itemcubso
adjudicaciones['codigogrupo'] = adjudicaciones['codigogrupo'].astype('Int64')
adjudicaciones['codigoitem'] = adjudicaciones['codigoitem'].astype('Int64')
adjudicaciones['codigofamilia'] = adjudicaciones['codigofamilia'].astype('Int64')
adjudicaciones['codigoclase'] = adjudicaciones['codigoclase'].astype('Int64')
adjudicaciones['codigocommodity'] = adjudicaciones['codigocommodity'].astype('Int64')

convocatorias['codigo_grupo'] = convocatorias['codigo_grupo'].astype('Int64')
convocatorias['codigoitem'] = convocatorias['codigoitem'].astype('Int64')
convocatorias['codigo_familia'] = convocatorias['codigo_familia'].astype('Int64')
convocatorias['codigoclase'] = convocatorias['codigoclase'].astype('Int64')
convocatorias['codigocommodity'] = convocatorias['codigocommodity'].astype('Int64')

In [30]:
# Funcion paraagregar 00 adelante de codigoitem para que tenga 8 caracteres de largo
def agregar_ceros_codigoitem(codigoitem):
    if pd.isnull(codigoitem):
        nuevo_codigoitem = codigoitem
    else:
        if len(str(codigoitem)) == 1:
            nuevo_codigoitem = str("0000000") + str(codigoitem)
        elif len(str(codigoitem)) == 2:
            nuevo_codigoitem = str("000000") + str(codigoitem)
        elif len(str(codigoitem)) == 3:
            nuevo_codigoitem = str("00000") + str(codigoitem)
        elif len(str(codigoitem)) == 4:
            nuevo_codigoitem = str("0000") + str(codigoitem)
        elif len(str(codigoitem)) == 5:
            nuevo_codigoitem = str("000") + str(codigoitem)
        elif len(str(codigoitem)) == 6:
            nuevo_codigoitem = str("00") + str(codigoitem)
        elif len(str(codigoitem)) == 7:
            nuevo_codigoitem = str("0") + str(codigoitem)
        else:
            nuevo_codigoitem = str(codigoitem)
        nuevo_codigoitem = nuevo_codigoitem[0:8]
    return (nuevo_codigoitem)

In [31]:
# Agrego 00 adelante de codigoitem para que tenga 8 caracteres de largo
adjudicaciones['codigoitem'] = adjudicaciones.apply(lambda x : agregar_ceros_codigoitem(x['codigoitem']) 
                                      if x['codigoitem'] is not None else x['codigoitem'], axis = 1)

convocatorias['codigoitem'] = convocatorias.apply(lambda x : agregar_ceros_codigoitem(x['codigoitem']) 
                                      if x['codigoitem'] is not None else x['codigoitem'], axis = 1)

In [32]:
convocatorias['codigoitem'] = convocatorias['codigoitem'].where(pd.notnull(convocatorias['codigoitem']),None)
adjudicaciones['codigoitem'] = adjudicaciones['codigoitem'].where(pd.notnull(adjudicaciones['codigoitem']),None)

In [33]:
# Concatenar codigos items en convocatorias
convocatorias['grupo_familia'] = convocatorias['codigo_grupo'].astype(str) +'-'+ convocatorias['codigo_familia'].astype(str)
convocatorias['grupo_familia_clase'] = convocatorias['codigo_grupo'].astype(str) +'-'+ convocatorias['codigo_familia'].astype(str) +'-'+ convocatorias['codigoclase'].astype(str)
convocatorias['grupo_familia_clase_commodity'] = convocatorias['codigo_grupo'].astype(str) +'-'+ convocatorias['codigo_familia'].astype(str) +'-'+ convocatorias['codigoclase'].astype(str)+'-'+ convocatorias['codigocommodity'].astype(str)

In [34]:
# Concatenar codigos items en adjudicaciones
adjudicaciones['grupo_familia'] = adjudicaciones['codigogrupo'].astype(str) +'-'+ adjudicaciones['codigofamilia'].astype(str)
adjudicaciones['grupo_familia_clase'] = adjudicaciones['codigogrupo'].astype(str) +'-'+ adjudicaciones['codigofamilia'].astype(str) +'-'+ adjudicaciones['codigoclase'].astype(str)
adjudicaciones['grupo_familia_clase_commodity'] = adjudicaciones['codigogrupo'].astype(str) +'-'+ adjudicaciones['codigofamilia'].astype(str) +'-'+ adjudicaciones['codigoclase'].astype(str)+'-'+ adjudicaciones['codigocommodity'].astype(str)

In [35]:
# DGR
# Renombro columnas
dgr_aso = dgr_aso.rename(columns = {'fecha\noficio':'fecha_oficio',
                                   'n° de expediente (seace)': 'codigoconvocatoria'})

dgr_dict = dgr_dict.rename(columns = {'fecha de emisión':'fecha_emision',
                                   'conclusión \nfinal': 'conclusion_final',
                                   'nº de expediente - seace': 'codigoconvocatoria'})


In [36]:
# flatten de dataframe para vincular ambas tablas(match)
match_sin_duplicado = match.drop_duplicates(['año_de_convocatoria','ruc_entidad','numero_ref_pac'])#[['año_de_convocatoria','ruc_entidad','numero_ref_pac','nomenclatura','año_pac']]

In [37]:
# datatypes
convocatorias_agg['anio'] = convocatorias_agg['anio'].astype(str)
convocatorias_agg['ruc_entidad'] = convocatorias_agg['ruc_entidad'].astype(str)
convocatorias_agg['proceso'] = convocatorias_agg['proceso'].astype(str)

match_sin_duplicado['año_de_convocatoria'] = match_sin_duplicado['año_de_convocatoria'].astype(str)
match_sin_duplicado['año_pac'] = match_sin_duplicado['año_pac'].astype(str)
match_sin_duplicado['ruc_entidad'] = match_sin_duplicado['ruc_entidad'].astype(str)
match_sin_duplicado['nomenclatura'] = match_sin_duplicado['nomenclatura'].astype(str)

pac_agg['año'] = pac_agg['año'].astype(str)
pac_agg['ruc_entidad'] = pac_agg['ruc_entidad'].astype(str)
pac_agg['n_referencia'] = pac_agg['n_referencia'].astype(int)

<ipython-input-37-b8f9bb1774a9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_sin_duplicado['año_de_convocatoria'] = match_sin_duplicado['año_de_convocatoria'].astype(str)
<ipython-input-37-b8f9bb1774a9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_sin_duplicado['año_pac'] = match_sin_duplicado['año_pac'].astype(str)
<ipython-input-37-b8f9bb1774a9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [38]:
# Reemplazar nulls por ceros
postores_agg['cant_ruc_codigo_postor'].fillna(0, inplace = True)
participantes_agg['cant_rucparticipante'].fillna(0, inplace = True)

In [39]:
# Agrego una columna a cada dataframe para identificar el estadio
convocatorias_agg['convocada'] = 'Si'
adjudicaciones_agg['adjudicada'] = 'Si'
contratos_agg['contrato'] = 'Si'

In [40]:
# Contratos: Genero columna en la que si Ruc ganador de pago es consorcio, tomo ruc destinatario de pago
contratos['ruc_proveedor'] = contratos.apply(lambda x: x['ruc_destinatario_pago'] if x['es_consorcio'] == 'S'
                                            else x['ruc_codigo_ganador'], axis = 1)
# Contratos: Genero columna en la que si Ruc ganador de pago es consorcio, tomo ruc destinatario de pago
contratos['ruc_proveedor_razon_social'] = contratos.apply(lambda x: x['nombre_destinatario_pago'] if x['es_consorcio'] == 'S'
                                            else x['ganador'], axis = 1)

In [42]:
def solo_alfanumericos(texto):
    if texto is None:
        resultado = None
    else:
        resultado = re.sub(r'[^A-Za-z\u00f1\u00d1]',' ', texto)
    return(resultado)

In [43]:
# Sacar caracteres especiales de razon social
adjudicaciones['ganador2'] = adjudicaciones.apply(lambda x: solo_alfanumericos(x['ganador']), axis = 1)

In [46]:
# Sacar caracteres especiales de razon social
contratos['ruc_proveedor_razon_social'] = contratos.apply(lambda x: solo_alfanumericos(x['ruc_proveedor_razon_social']), axis = 1)

In [47]:
# Sacar caracteres especiales de razon social
postores['detalle_postor'] = postores.apply(lambda x: solo_alfanumericos(x['detalle_postor']), axis = 1)

In [48]:
# Agrego a tabla convocatoria, datos de convocatoria(fechas) que teniamos en Datos abiertos y en nueva base no estan
convocatorias_old_agg = convocatorias_old_agg[['codigoconvocatoria','fechaintegracionbases','fechapresentacionpropuesta','montoreferencial']]

# Join de data nueva con data de datos abiertos para traernos las fechas de integracion y presentacion de propuestas
convocatorias_agg = convocatorias_agg.join(convocatorias_old_agg.set_index(['codigoconvocatoria']),
             on = ['codigoconvocatoria'],                                       
             how = 'left')

In [49]:
# Join de tabla convocatorias con tabla para join (match)
convo_match = convocatorias_agg[['codigoconvocatoria','anio','ruc_entidad','proceso']].join(match_sin_duplicado\
                                .set_index(['año_de_convocatoria','ruc_entidad','nomenclatura']),
                                    on = ['anio','ruc_entidad','proceso'],
                                    how = 'inner')

In [50]:
# Borro duplicados que surgen de que exista el mismo proceso para el mismo ruc y anio pero distinto nro ref pac
convo_match.drop_duplicates('codigoconvocatoria', inplace=True)

In [51]:
# Join de tabla convocatorias con tabla para join (match) con pac
convo_match_pac = convo_match.join(pac_agg.set_index(['año','ruc_entidad','n_referencia']),
                        on = ['año_pac','ruc_entidad','numero_ref_pac'],
                        how = 'inner')

In [52]:
# Selecciono columnas
convo_match_pac = convo_match_pac[['codigoconvocatoria','ruc_entidad','anio','proceso','numero_ref_pac','año_pac','max_version','tipoprocesoseleccion','max_fecha_publicacion']]

In [53]:
# Borro 1 duplicado
convo_match_pac.drop_duplicates('codigoconvocatoria', inplace=True)

In [54]:
# Agrego una columna a cada dataframe para identificar el estadio
convo_match_pac['pac'] = 'Si'

#### Malla PAC-Convocatoria-Ajudicaciones-Contratos

In [56]:
# Join pac con convocatorias
df_pac_conv = convo_match_pac.join(convocatorias_agg.set_index('codigoconvocatoria'),
                               on = 'codigoconvocatoria',
                               how = 'left',
                               rsuffix = '_conv')

In [57]:
# Join convocatoria con adjudicaciones
df_agg = df_pac_conv.join(adjudicaciones_agg.set_index('codigoconvocatoria'),
                               on = 'codigoconvocatoria',
                               how = 'left',
                               rsuffix = '_adj')

In [58]:
# Saco convocatorias sin adjudicaciones 
df_agg_ca = df_agg.loc[~df_agg['ruc_entidad_adj'].isna()]

In [59]:
# Join convocatoria con adjudicaciones con contratos
df_agg_pcac = df_agg_ca.join(contratos_agg.set_index('codigoconvocatoria'),
                      on = 'codigoconvocatoria',
                      how = 'left',
                      rsuffix = '_contr')

In [60]:
# Borro duplicados 2.989
df_agg_pcac.drop_duplicates('codigoconvocatoria', inplace=True)

#### Malla Convocatoria-Ajudicaciones-Contratos

In [62]:
# Join convocatoria con adjudicaciones
df_agg = convocatorias_agg.join(adjudicaciones_agg.set_index('codigoconvocatoria'),
                               on = 'codigoconvocatoria',
                               how = 'left',
                               rsuffix = '_adj')

In [63]:
# Saco convocatorias sin adjudicaciones 
df_agg_ca = df_agg.loc[~df_agg['ruc_entidad_adj'].isna()]

In [64]:
# Join convocatoria con adjudicaciones con contratos
df_agg_cac = df_agg_ca.join(contratos_agg.set_index('codigoconvocatoria'),
                      on = 'codigoconvocatoria',
                      how = 'left',
                      rsuffix = '_contr')

In [65]:
# Borro duplicados 2.989
df_agg_cac.drop_duplicates('codigoconvocatoria', inplace=True)

In [66]:
# Genero campo de monto de convocatoria
# Si tengo el dato en tabla contrtos uso ese (ultimo envio de BD), caso contrario lo traigo de
# tabla convocatorias de base de datos anterior
df_agg_cac['monto_convocatoria'] = df_agg_cac.apply(lambda x: x['montoreferencial'] if pd.isnull(x['monto_referencial_total'])
                                                   else x['monto_referencial_total'], axis = 1)




#### Universo de análisis

In [68]:
# Lista procesos seleccionados 
tipoprocesos = ['Adjudicación Simplificada', 
                'Contratación Directa', 
                'Subasta Inversa Electrónica',
                'Licitación Pública',
                'Comparación de Precios', 
                'Concurso Público']


# Saco los tipos de contrataciones que no vamos a utilizar - Malla que incluye PAC
df_agg_pcac = df_agg_pcac[df_agg_pcac['tipoprocesoseleccion'].isin(tipoprocesos)]
df_agg_pcac = df_agg_pcac[df_agg_pcac['tipoprocesoseleccion'].isin(tipoprocesos)]
print(f'Cantidad de filas al seleccionar procesos - malla PAC-Conv-Adj-Contr: {df_agg_pcac.shape[0]}')

# Saco los tipos de contrataciones que no vamos a utilizar - Malla sin PAC
df_agg_cac = df_agg_cac[df_agg_cac['tipoprocesoseleccion'].isin(tipoprocesos)]
df_agg_cac = df_agg_cac[df_agg_cac['tipoprocesoseleccion'].isin(tipoprocesos)]
print(f'Cantidad de filas al seleccionar procesos - malla Conv-Adj-Contr: {df_agg_cac.shape[0]}')

Cantidad de filas al seleccionar procesos - malla PAC-Conv-Adj-Contr: 114141
Cantidad de filas al seleccionar procesos - malla Conv-Adj-Contr: 143735


## 4. Banderas

### Bandera 1: Número de versiones de PAC es demasiado alto

In [69]:
# Start time de bandera
start = datetime.datetime.now()

In [70]:
# Genero lista con rucs de entidades pertenecientes a cada grupo
tipoa = list(entes_tipoa['RUC de la Entidad'])
tipob = list(entes_tipob['RUC de la Entidad'])
tipoc = list(entes_tipoc['RUC de la Entidad'])
tipod = list(entes_tipod['RUC de la Entidad'])
tipoe = list(entes_tipoe['RUC de la Entidad'])

In [71]:
df_agg_pcac['ruc_entidad'] = df_agg_pcac['ruc_entidad'].astype(int)

In [72]:
# Defino funcion para asignar grupo a cada entidad
def grupo_entidad(ruc_entidad,tipoa,tipob,tipoc,tipod,tipoe):
    """
    Devuelve el grupo al que pertenece un RUC de entidad
    
    Params
    ------
    ruc_entidad(integer): RUC de la entidad
    tipoa(list): Listado de RUCs de entidad pertenecientes al grupo A
    tipob(list): Listado de RUCs de entidad pertenecientes al grupo B
    tipoc(list): Listado de RUCs de entidad pertenecientes al grupo C
    tipod(list): Listado de RUCs de entidad pertenecientes al grupo D
    tipoe(list): Listado de RUCs de entidad pertenecientes al grupo E
    
    Returns
    -------
    resultado(string): Letra del grupo al que pertenece la entidad
    
    """
    if ruc_entidad in tipoa:
        resultado = 'A'
    elif ruc_entidad in tipob:
        resultado = 'B'
    elif ruc_entidad in tipoc:
        resultado = 'C'
    elif ruc_entidad in tipod:
        resultado = 'D'
    elif ruc_entidad in tipoe:
        resultado = 'E'
    else:
        resultado = None
    return(resultado)

In [73]:
# Asigno grupo a la entidad - en df de pacs
df_agg_pcac['grupo_entidad'] = df_agg_pcac.apply(lambda x: grupo_entidad(x['ruc_entidad'],tipoa,tipob,tipoc,tipod,tipoe), axis = 1)

In [74]:
# Selecciono columnas relevantes para la bandera a construir
df_bandera_1 = df_agg_pcac[['anio','codigoconvocatoria','fecha_convocatoria','ruc_entidad','grupo_entidad','entidad','max_version']]

In [75]:
# Me quedo solo con 2021 para que tarde menos en correr el proceso
df_bandera_1 = df_bandera_1.loc[df_bandera_1['anio'] == '2021']

In [76]:
# Asigno grupo a la entidad - en df de bandera
df_bandera_1['grupo_entidad'] = df_bandera_1.apply(lambda x: grupo_entidad(x['ruc_entidad'],tipoa,tipob,tipoc,tipod,tipoe), axis = 1)


In [77]:
# Selecciono anio de presentacion de PAC a considerar
def seleccion_anio_pac(df,anio):
    '''
    Filtra dataframe que contiene data de PAC, filtrando por el anio de presentacion de PAC
    
    Params
    ------
    df (dataframe): Dataframe que contiene anio de presentacion de PAC
    anio (string): Anio en que se presento el PAC
    
    Returns
    ------
    df (dataframe) : Dataframe filtrado para el anio que se quiere analizar
    '''
    
    df = df.loc[df['anio'] == anio]
    return(df)

In [78]:
# Me quedo solo con las PAC del anio que quiero considerar
df_agg_pcac_2020 = seleccion_anio_pac(df_agg_pcac,'2020')

In [79]:
# Calculo media y std para cada grupo
grupos_desc = df_agg_pcac_2020.groupby('grupo_entidad',as_index = False)[['grupo_entidad','max_version']].agg(['mean','std'])

In [80]:
# Flatten del hierarchie column index
grupos_desc.columns = [' '.join(col).strip() for col in grupos_desc.columns.values]

In [81]:
# Convierto index a columna (grupo)
grupos_desc.reset_index(level=0, inplace=True)

In [82]:
# Renombro columnas
grupos_desc = grupos_desc.rename(columns = {'max_version mean':'bandera_1_media_versionesPAC',
                             'max_version std': 'bandera_1_std_versionesPAC'})

In [83]:
# Calculo umbral(media + 2 * std)
grupos_desc['bandera_1_umbral_versionesPAC'] = grupos_desc['bandera_1_media_versionesPAC'] + 2 * grupos_desc['bandera_1_std_versionesPAC']

In [84]:
# Join de df bandera con desc por grupo
df_bandera_1 = df_bandera_1.join(grupos_desc.set_index('grupo_entidad'),
                                  on = 'grupo_entidad',
                                  how = 'left')

In [85]:
# Genero bandera
df_bandera_1['bandera_1'] = df_bandera_1.apply(lambda x: 'Si' if (x['max_version'] > x['bandera_1_umbral_versionesPAC'])
                                              else 'No', axis = 1)

In [86]:
# Descriptive stats con decimales
df_bandera_1[['bandera_1_media_versionesPAC','bandera_1_std_versionesPAC','bandera_1_umbral_versionesPAC']] = df_bandera_1[['bandera_1_media_versionesPAC','bandera_1_std_versionesPAC','bandera_1_umbral_versionesPAC']].applymap("{0:.2f}".format)

In [87]:
# Renombro columnas
df_bandera_1.rename(columns = {'max_version': 'bandera_1_version', 'grupo_entidad':'bandera_1_grupo_entidad'},
                   inplace = True)

In [88]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('1',start,end, tiempos)

### Bandera 2: Un ente tiene una alta proporción de contratación directa

In [97]:
# Start time de bandera
start = datetime.datetime.now()

In [98]:
# Selecciono variables relevantes para la bandera
df_bandera2 = df_agg_cac[['codigoconvocatoria','anio','fechaconvocatoria','tipoprocesoseleccion','ruc_entidad']]

In [99]:
# Solo 2021
df_bandera2 = df_bandera2.loc[df_bandera2['anio'] == '2021']

In [100]:
# Solo contratacion directa
df_bandera2_cd = df_bandera2.loc[df_bandera2['tipoprocesoseleccion'] == 'Contratación Directa']
df_bandera2_resto = df_bandera2.loc[df_bandera2['tipoprocesoseleccion'] != 'Contratación Directa']

In [101]:
def proporcion_contrataciondirecta_itemcubso_ente(codigoconvocatoria,fechaconvocatoria, ruc_entidad,convocatorias, months = 12):
    # 1. Genero lista con items de la convocatoria analizada - excluyendo codigoitem null
    items_convocatoria = convocatorias.loc[(convocatorias['codigoconvocatoria'] == codigoconvocatoria) & (pd.notnull(convocatorias['codigoitem']))]['codigoitem'].unique()
    if len(items_convocatoria) == 0:
        diccionario = {}
        diccionario[f'Entidad {ruc_entidad}'] = f'Codigos de item cubso nulos'
    else:
        diccionario = {}
        # 2. Filtro tabla convocatoria para ventana temporal a analizar
        fecha_inicio = fechaconvocatoria - pd.DateOffset(months=months)
        df = convocatorias.loc[(convocatorias['fecha_convocatoria'] < fechaconvocatoria) & (convocatorias['fecha_convocatoria'] > fecha_inicio)] # filtro ventana temporal
        # 3. Check si la entidad esta en la ventana temporal
        ruc_entidad = int(ruc_entidad)
        df_entidad = df.loc[df['ruc_entidad'] == ruc_entidad]
        
        if df_entidad.empty: #si la entidad no esta en las convocatorias de ultimos 12 meses
            diccionario = {}
            diccionario[f'Entidad {ruc_entidad}'] = 'Entidad no tiene convocatorias en la ventana consultada'

        else: # Si la entidad esta en las conv de ult 12 meses, recorro cada item de la convocatoria
            for item in items_convocatoria:
                # Filtro df de convocatorias de ventana para ver el item en analisis
                df_item = df.loc[df['codigoitem'] == item]
                # Calculo cantidad de convocatorias para cada entidad y proceso de seleccion
                agg_df = df_item.groupby(['ruc_entidad','tipoprocesoseleccion'],as_index = False).count()[['ruc_entidad','tipoprocesoseleccion','codigoconvocatoria']].rename(columns = {'codigoconvocatoria':'cantidad'})
                # Calculo % que cada proceso de seleccion representa para cada entidad
                agg_df['porcentaje'] = (agg_df.cantidad*1.0 / agg_df.cantidad.sum())*100
                # Me quedo solo con los procesos de contratacion directa
                agg_df_contratacion_directa = agg_df.loc[agg_df['tipoprocesoseleccion'] == 'Contratación Directa']
                # Me quedo solo con los procesos de contratacion directa de este ente
                agg_df_contratacion_directa_ente = agg_df.loc[agg_df['ruc_entidad'] == ruc_entidad]
                if  agg_df_contratacion_directa.empty: # No hay convocatorias de ese item
                    diccionario[f'itemcubso {item}'] = f'Entidad no tiene convocatorias de item {item} en la ventana consultada'
                elif agg_df_contratacion_directa_ente.empty: # No hay contratcion directa
                    diccionario[f'itemcubso {item}'] = f'Entidad no tiene convocatorias de contratación directa del item {item} en la ventana consultada'
                elif min(agg_df_contratacion_directa['porcentaje']) >= 90: # mercado no competitivo - 
                    diccionario[f'itemcubso {item}'] = f'Item {item} Mercado no competitivo'
                else:
                    media = agg_df_contratacion_directa['porcentaje'].mean()
                    std = agg_df_contratacion_directa['porcentaje'].std() 
                    umbral = agg_df_contratacion_directa['porcentaje'].mean() + 2 * agg_df_contratacion_directa['porcentaje'].std()  
                    porcentaje_ente = agg_df_contratacion_directa_ente['porcentaje'].values[0]
                    if porcentaje_ente > umbral:
                        diccionario[f'itemcubso {item}'] = {'Media': round(media,2), 'Std': round(std,2), 'Umbral': round(umbral,2), 'Porcentaje_ente': round(porcentaje_ente,2)}
                    else:
                        diccionario[f'itemcubso {item}'] = 'Ok: Porcentaje < +2STD'
    return (diccionario)
        

In [102]:
df_bandera2_cd['bandera2_ente_%contrdirecta_item_mayor_2SD']= df_bandera2_cd.apply(lambda x: proporcion_contrataciondirecta_itemcubso_ente(x['codigoconvocatoria'],
                                                                                                               x['fechaconvocatoria'], 
                                                                                                               x['ruc_entidad'],
                                                                                                               convocatorias, 
                                                                                                               months = 12)
                                                       , axis = 1)



<ipython-input-102-a0a487faf8aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera2_cd['bandera2_ente_%contrdirecta_item_mayor_2SD']= df_bandera2_cd.apply(lambda x: proporcion_contrataciondirecta_itemcubso_ente(x['codigoconvocatoria'],


In [103]:
def generar_bandera_2 (atributo):
    atributo = str(atributo.values())
    if 'Porcentaje_ente' in atributo:
        resultado = 'Si'
    elif 'Codigos de item cubso nulos' in atributo:
        resultado = 'No aplica'
    elif 'Entidad no tiene' in atributo:
        resultado = 'No aplica'
    elif 'Mercado no competitivo' in atributo:
        resultado = 'No aplica'
    elif 'Ok:' in atributo:
        resultado = 'No'
    else:
        resultado = None
    return(resultado)

In [104]:
# Genero bandera
df_bandera2_cd['bandera_2'] = df_bandera2_cd.apply(lambda x: generar_bandera_2(x['bandera2_ente_%contrdirecta_item_mayor_2SD']), axis = 1)

<ipython-input-104-2ce31e9aaed2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera2_cd['bandera_2'] = df_bandera2_cd.apply(lambda x: generar_bandera_2(x['bandera2_ente_%contrdirecta_item_mayor_2SD']), axis = 1)


In [105]:
# Los casos ok pongo en null el atributo
df_bandera2_cd['bandera2_ente_%contrdirecta_item_mayor_2SD'] = df_bandera2_cd.apply(lambda x: None if x['bandera_2'] == 'No' else x['bandera2_ente_%contrdirecta_item_mayor_2SD'], axis = 1)

<ipython-input-105-30e59fb3e5cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera2_cd['bandera2_ente_%contrdirecta_item_mayor_2SD'] = df_bandera2_cd.apply(lambda x: None if x['bandera_2'] == 'No' else x['bandera2_ente_%contrdirecta_item_mayor_2SD'], axis = 1)


In [106]:
# Concat con resto de procesos
df_bandera2_resto['bandera2_ente_%contrdirecta_item_mayor_2SD'] = None
df_bandera2_resto['bandera_2'] = 'No aplica'

df_bandera2 = pd.concat([df_bandera2_cd,df_bandera2_resto])

<ipython-input-106-1ad581140bb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera2_resto['bandera2_ente_%contrdirecta_item_mayor_2SD'] = None
<ipython-input-106-1ad581140bb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera2_resto['bandera_2'] = 'No aplica'


In [107]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('2',start,end, tiempos)

### Bandera 3: El número de días entre  fecha de convocatoria y fecha de presentación de propuestas es inferior al marcado en legislación

In [118]:
# Start time de bandera
start = datetime.datetime.now()

In [119]:
# Selecciono columnas relevantes para la bandera a construir
df_bandera_3 = df_agg_cac[['anio','codigoconvocatoria','tipoprocesoseleccion','fechaconvocatoria','fechapresentacionpropuesta']]

# Calculo diferencia de dias entre fechas
df_bandera_3['fechapresentacionpropuesta'].fillna('1900-01-01', inplace=True) # nan a 1900 para poder computar np.busday
df_bandera_3['fechaconvocatoria'].fillna('1900-01-01', inplace=True) # nan a 1900 para poder computar np.busday

# Calculo diferencia de dias entre fechas excluyendo fin de semanas y feriados
df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] = np.busday_count(df_bandera_3['fechaconvocatoria'].values.astype('datetime64[D]'),
                                       df_bandera_3['fechapresentacionpropuesta'].values.astype('datetime64[D]'), 
                                      holidays = fecha_feriados)

# Asigno NA a los casos en los que fecha convocatoria y/o fechapresentacionpropuesta es null
df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] = df_bandera_3.apply(lambda x: None if x['fechapresentacionpropuesta'] == '1900-01-01' or x['fechaconvocatoria'] == '1900-01-01'
                                                                                      else x['bandera_3_dif_convocatoria_presentacionpropuestas'], axis = 1)

/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-119-60681d7a4748>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] = np.busday_count(df_bandera_3['fechaconvocatoria'].values.astype('datetime64[D]'),
<ipython-input-119-60681d7a4748>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [120]:
# Defino cantidad de dias de acuerdo a tipo de proceso
def bandera_3_umbral_dias(tipoprocesoseleccion):
    """
    Determina cantidad de dias máximos que pueden transcurrir entre fecha de convocatoria y fecha de presentación de propuesta
    de acuerdo al tipo de proceso de selección
    
    Params
    ------
    tipoprocesoseleccion(string): Tipo de proceso de seleccion de la convocatoria, por ejemplo 'Adjudicación Simplificada'
    
    Returns
    -------
    Cantidad de días máximos que pueden transcurrir entre fecha de convocatoria y fecha de presentación de propuesta
    dado el tipo de proceso de selección
    
    """
    if tipoprocesoseleccion == 'Adjudicación Simplificada':
        resultado = 3
    elif tipoprocesoseleccion == 'Licitación Pública':
        resultado = 22
    elif tipoprocesoseleccion == 'Concurso Público':
        resultado = 22
    else:
        resultado = None
    return(resultado)

In [121]:
# Agrego columna que indica umbral
df_bandera_3['bandera_3_umbral'] = df_bandera_3.apply(lambda x: bandera_3_umbral_dias(x['tipoprocesoseleccion']), axis = 1)

<ipython-input-121-896b831228b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_3['bandera_3_umbral'] = df_bandera_3.apply(lambda x: bandera_3_umbral_dias(x['tipoprocesoseleccion']), axis = 1)


In [122]:
# listo los procesos a los que no aplica ningun umbral
procesos_noaplica = ['Contratación Directa','Subasta Inversa Electrónica','Comparación de Precios']

In [123]:
# Computo bandera 3
df_bandera_3['bandera_3'] = df_bandera_3.apply(lambda x: 'Si' 
                                               if x['bandera_3_dif_convocatoria_presentacionpropuestas'] < x['bandera_3_umbral'] 
                                               else ('No aplica' if x['tipoprocesoseleccion'] in procesos_noaplica else'No'), 
                                               axis = 1)

<ipython-input-123-37caa7c4e0d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_3['bandera_3'] = df_bandera_3.apply(lambda x: 'Si'


In [124]:
# Asigno No aplica al campo de bandera para los casos en que presentacion propuesta es de 1900
df_bandera_3['bandera_3'] = np.where(df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] < -10000,
                                     'No aplica', df_bandera_3['bandera_3'])

# Asigno null al campo de atributo para los casos en que presentacion propuesta es de 1900
df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] = np.where(df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] < -10000,
                                     None, df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'])


<ipython-input-124-c5455144e752>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_3['bandera_3'] = np.where(df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] < -10000,
<ipython-input-124-c5455144e752>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] = np.where(df_bandera_3['bandera_3_dif_convocatoria_presentacionpropuestas'] < -10000,


In [126]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('3',start,end, tiempos)

### Bandera 3a: El número de días entre fecha de convocatoria y fecha de presentación de propuestas es demasiado largo

In [135]:
# Start time de bandera
start = datetime.datetime.now()

In [136]:
# Selecciono columnas relevantes para la bandera a construir
df_bandera_3a = df_agg_cac[['anio','codigoconvocatoria','tipoprocesoseleccion','fechaconvocatoria','fechapresentacionpropuesta']]

# Calculo diferencia de dias entre fechas
df_bandera_3a['fechapresentacionpropuesta'].fillna('1900-01-01', inplace=True) # nan a 1900 para poder computar np.busday
df_bandera_3a['fechaconvocatoria'].fillna('1900-01-01', inplace=True) # nan a 1900 para poder computar np.busday

# Calculo diferencia de dias entre fechas excluyendo fin de semanas y feriados
df_bandera_3a['bandera_3a_dif_convocatoria_presentacionpropuestas'] = np.busday_count(df_bandera_3a['fechaconvocatoria'].values.astype('datetime64[D]'),
                                       df_bandera_3a['fechapresentacionpropuesta'].values.astype('datetime64[D]'), 
                                      holidays = fecha_feriados)

# Asigno NA a los casos en los que fecha convocatoria y/o fechapresentacionpropuesta es null
df_bandera_3a['bandera_3a_dif_convocatoria_presentacionpropuestas'] = df_bandera_3a.apply(lambda x: None if x['fechapresentacionpropuesta'] == '1900-01-01' or x['fechaconvocatoria'] == '1900-01-01'
                                                                                      else x['bandera_3a_dif_convocatoria_presentacionpropuestas'], axis = 1)

# Reemplazo por null a la diferencia de dias negativa:
df_bandera_3a['bandera_3a_dif_convocatoria_presentacionpropuestas'] = df_bandera_3a.apply(lambda x: None if x['bandera_3a_dif_convocatoria_presentacionpropuestas'] < 0 
                                                                                      else x['bandera_3a_dif_convocatoria_presentacionpropuestas'], axis = 1)


/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-136-adef28ff2cd8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_3a['bandera_3a_dif_convocatoria_presentacionpropuestas'] = np.busday_count(df_bandera_3a['fechaconvocatoria'].values.astype('datetime64[D]'),
<ipython-input-136-adef28ff2cd8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [137]:
def estadisticas_dif_dias(fechaconvocatoria,tipoprocesoseleccion,df, months = 12):
    # Selecciono convocatorias de tipo de proceso de la convocatoria
    fecha_inicio = fechaconvocatoria - pd.DateOffset(months=months)
    df_convocatorias = df.loc[df['tipoprocesoseleccion']== tipoprocesoseleccion] # filtro proceso seleccion 
    df_convocatorias = df_convocatorias.loc[(df_convocatorias['fechaconvocatoria'] < fechaconvocatoria) & (df_convocatorias['fechaconvocatoria'] > fecha_inicio)] # filtro ventana temporal
    media = round(df_convocatorias['bandera_3a_dif_convocatoria_presentacionpropuestas'].mean(),2)
    std = round(df_convocatorias['bandera_3a_dif_convocatoria_presentacionpropuestas'].std(),2)
    umbral = round(df_convocatorias['bandera_3a_dif_convocatoria_presentacionpropuestas'].mean() + 2 * df_convocatorias['bandera_3a_dif_convocatoria_presentacionpropuestas'].std(),2)
    
    return(pd.Series([media,std,umbral]))

In [138]:
df_bandera_3a[['bandera_3a_media','bandera_3a_std','bandera_3a_umbral']] = df_bandera_3a.apply(lambda x: estadisticas_dif_dias(x['fechaconvocatoria'],
                                                                          x['tipoprocesoseleccion'],
                                                                          df_bandera_3a, 12), axis = 1)

/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [139]:
df_bandera_3a['bandera_3a'] = df_bandera_3a.apply(lambda x: 'Si' if x['bandera_3a_dif_convocatoria_presentacionpropuestas']> x['bandera_3a_umbral'] else 'No', axis = 1)

<ipython-input-139-dc580c265208>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_3a['bandera_3a'] = df_bandera_3a.apply(lambda x: 'Si' if x['bandera_3a_dif_convocatoria_presentacionpropuestas']> x['bandera_3a_umbral'] else 'No', axis = 1)


In [140]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('3a',start,end, tiempos)

### Bandera 4:  No se dispone de la información sobre el postor (la convocatoria no cuenta con postores) de una convocatoria adjudicada

In [149]:
# Start time de bandera
start = datetime.datetime.now()

In [150]:
# Creacion bandera
df_bandera_4 = df_agg_cac[['codigoconvocatoria','adjudicada']]\
               .join(postores_agg[['codigo_convocatoria','cant_ruc_codigo_postor']]\
               .set_index('codigo_convocatoria'),
                    on = 'codigoconvocatoria',
                    how = 'left')
df_bandera_4['bandera_4'] = df_bandera_4.apply(lambda x: "No" if x['cant_ruc_codigo_postor']>= 1 else "Si", axis = 1)
df_bandera_4 = df_bandera_4.rename(columns = {'cant_ruc_codigo_postor':'bandera_4_cantidad_ruc_postores',
                                             'adjudicada': 'bandera_4_adjudicada'})

In [151]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('4',start,end, tiempos)

### Bandera 4a: Número de participantes es mayor que el número de postores


In [152]:
# Start time de bandera
start = datetime.datetime.now()

In [153]:
# Creacion bandera
df_bandera_4a = df_agg_cac[['anio','codigoconvocatoria','fechaconvocatoria','tipoprocesoseleccion']]\
               .join(postores_agg[['codigo_convocatoria','cant_ruc_codigo_postor']]\
               .set_index('codigo_convocatoria'),
                    on = 'codigoconvocatoria',
                    how = 'left')\
               .join(participantes_agg[['codigoconvocatoria','cant_rucparticipante']]\
                    .set_index('codigoconvocatoria'),
                    on ='codigoconvocatoria',
                    how = 'left')

df_bandera_4a['ratio_postor_participante'] = round((df_bandera_4a['cant_ruc_codigo_postor'] * 1.00 / df_bandera_4a['cant_rucparticipante'] * 1.00 )* 100,2)
# 0 a los que no pudo hacer join
#df_bandera_4a.fillna(0,inplace = True) # reemplaz por null

In [154]:
def bandera4a_calcular_media_dosSD(df, tipoprocesoseleccion, fechaconvocatoria, months):
    fecha_inicio = fechaconvocatoria - pd.DateOffset(months=months)
    df = df.loc[df['tipoprocesoseleccion']== tipoprocesoseleccion] # filtro proceso seleccion 
    df = df.loc[(df['fechaconvocatoria'] < fechaconvocatoria) & (df['fechaconvocatoria'] > fecha_inicio)] # filtro ventana temporal
    
    media = round(df['ratio_postor_participante'].mean(),2)
    std = round(df['ratio_postor_participante'].std(),2)
    umbral = round(df['ratio_postor_participante'].mean() - 2 * df['ratio_postor_participante'].std(),2)
    return pd.Series([media,std,umbral])

In [155]:
# Solo 2021
df_bandera_4a_2021 = df_bandera_4a.loc[df_bandera_4a['anio'] == '2021']

In [156]:
# Calculo media y 2SD de "ratio postor participante" para cada tipo de proceso de seleccion 
# en los 365 dias previos a la fecha de convocatoria de la convocatoria analizada 
df_bandera_4a_2021[['bandera_4a_media_ratio_postor_participante','bandera_4a_std_ratio_postor_participante','bandera_4a_umbral_ratio_postor_participante']] = df_bandera_4a_2021.apply(lambda x: bandera4a_calcular_media_dosSD(df_bandera_4a, 
                                                                                                                                                                          x['tipoprocesoseleccion'],
                                                                                                                                                                          x['fechaconvocatoria'],
                                                                                                                                                                          12), 
                                                                                                                                 axis = 1)

/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [157]:
# Creo bandera
df_bandera_4a_2021['bandera_4a'] = df_bandera_4a_2021.apply(lambda x: 'Si' if x['ratio_postor_participante'] < x['bandera_4a_umbral_ratio_postor_participante'] else 'No', axis = 1)





<ipython-input-157-87e5c64168aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_4a_2021['bandera_4a'] = df_bandera_4a_2021.apply(lambda x: 'Si' if x['ratio_postor_participante'] < x['bandera_4a_umbral_ratio_postor_participante'] else 'No', axis = 1)


In [158]:
# Renombro columnas para identificar los atributos de la bandera
df_bandera_4a_2021 = df_bandera_4a_2021.rename(columns = {'cant_ruc_codigo_postor': 'bandera_4a_cant_ruc_codigo_postor',
                                    'cant_rucparticipante': 'bandera_4a_cant_rucparticipante',
                                    'ratio_postor_participante' : 'bandera_4a_ratio_postor_participante'})

In [160]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('4a',start,end, tiempos)

### Bandera 5: Se presenta un postor que nunca había ganado en un proceso de compras públicas antes

In [169]:
# Start time de bandera
start = datetime.datetime.now()

In [170]:
# Selecciono variables relevantes para la bandera
df_bandera5 = df_agg_cac[['codigoconvocatoria','tipoprocesoseleccion','anio','fechaconvocatoria']]

In [171]:
def busqueda_ruc_tabla_adjudicaciones(codigoconvocatoria,fechaconvocatoria,df_contratos):
    '''
    Se buscan los RUCS de los proveedores ganadores de una convocatoria en adjudicaciones previas a la fecha
    de convocatoria de la convocatoria analizada
    
    Params
    ------
    codigoconvocatoria (string): Número de identificación de convocatoria analizada
    fechaconvocatoria(datetime): Fecha de convocatoria analizada
    df_contratos(dataframe): Dataframe que contiene todas las contrataciones
    
    Returns
    ------
    proveedores_primera_vez(list): Lista de todos los proveedores que ganaron la convocatoria analizada y que
    no habian ganado un proceso de contratacion antes 
    '''
    df_contrato = df_contratos.loc[df_contratos['codigoconvocatoria'] == codigoconvocatoria]
    proveedores_de_convocatoria = df_contrato['ruc_proveedor'].unique()
    proveedores_de_convocatoria_razon_social = df_contrato['ruc_proveedor_razon_social'].unique()
    proveedores_primera_vez = []
    for proveedor,proveedor_razonsocial in zip(proveedores_de_convocatoria,proveedores_de_convocatoria_razon_social):
        # 1.Filtro contratos anteriores a la fecha de convocatoria
        df_contratos = df_contratos.loc[df_contratos['fechaconvocatoria'] < fechaconvocatoria]
        # 2. Me quedo con listado de proveedores 
        proveedores = df_contratos['ruc_codigo_ganador'].unique()
        # 3. Busco proveedor en proveedores
        if proveedor in proveedores:
            continue
        else:
            proveedores_primera_vez.append((proveedor,proveedor_razonsocial))
            #proveedores_primera_vez.append(proveedor)
    return(proveedores_primera_vez)

In [172]:
# Solo contratacion directa y 2021
df_bandera5_condir_2021 = df_bandera5.loc[(df_bandera5['tipoprocesoseleccion']=='Contratación Directa') & (df_bandera5['anio']=='2021')]
df_bandera5_resto=pd.concat([df_bandera5,df_bandera5_condir_2021]).drop_duplicates(keep=False)

In [173]:
# Atributo: proveedores primera vez
df_bandera5_condir_2021['bandera_5_proveedores_primera_vez'] = df_bandera5_condir_2021.apply(lambda x: busqueda_ruc_tabla_adjudicaciones(x['codigoconvocatoria'], 
                                                                                                                                      x['fechaconvocatoria'],
                                                                                                                                      contratos),
                                                                                                          axis = 1)

<ipython-input-173-4c922812d985>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera5_condir_2021['bandera_5_proveedores_primera_vez'] = df_bandera5_condir_2021.apply(lambda x: busqueda_ruc_tabla_adjudicaciones(x['codigoconvocatoria'],


In [174]:
# Cambio list vacia por null
df_bandera5_condir_2021['bandera_5_proveedores_primera_vez'] = df_bandera5_condir_2021.apply(lambda x: None if len(x['bandera_5_proveedores_primera_vez']) == 0 else x['bandera_5_proveedores_primera_vez'], axis = 1)

<ipython-input-174-bf082bfa2172>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera5_condir_2021['bandera_5_proveedores_primera_vez'] = df_bandera5_condir_2021.apply(lambda x: None if len(x['bandera_5_proveedores_primera_vez']) == 0 else x['bandera_5_proveedores_primera_vez'], axis = 1)


In [175]:
# Genero bandera
df_bandera5_condir_2021['bandera_5'] = df_bandera5_condir_2021.apply(lambda x: 'No' if x['bandera_5_proveedores_primera_vez'] is None else 'Si', axis = 1)

<ipython-input-175-18b65532ff30>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera5_condir_2021['bandera_5'] = df_bandera5_condir_2021.apply(lambda x: 'No' if x['bandera_5_proveedores_primera_vez'] is None else 'Si', axis = 1)


In [176]:
# agrego al dataset los procesos que no son contratacion directa o 2021
df_bandera5_resto['bandera_5_proveedores_primera_vez'] = 'No aplica'
df_bandera5_resto['bandera_5'] = 'No aplica'

In [177]:
# Join de dataframe con bandera par 2021 condirecta y resto
df_bandera5 = pd.concat([df_bandera5_condir_2021,df_bandera5_resto])

In [178]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('5',start,end, tiempos)

### Bandera 5a: Se presenta un postor que nunca había ganado en un proceso de compras públicas para este grupo de ítems

In [184]:
# Start time de bandera
start = datetime.datetime.now()

In [185]:
# Selecciono variables relevantes para la bandera
df_bandera5a = df_agg_cac[['codigoconvocatoria','anio','fechaconvocatoria']]

In [186]:
# Solo 2021
df_bandera5a_2021 = df_bandera5a.loc[df_bandera5a['anio']=='2021']

In [187]:
# Genero tabla con codigoconvocatoria y postulantes unicos 
# (en tabla original esta aperturado por item y duplica postor)
postores_cc = postores.groupby(['codigo_convocatoria','ruc_detallepostor','detalle_postor'], as_index =  False).count()

In [188]:
# Renombro columnas de items en adjudicaciones para que sean iguales que en convocatorias
df_adjudicaciones = adjudicaciones.copy()
df_adjudicaciones['codigo_grupo'] = df_adjudicaciones['codigogrupo']
df_adjudicaciones['grupo'] = df_adjudicaciones['descripcion_grupo']
df_adjudicaciones['familia'] = df_adjudicaciones['descripcion_familia']
df_adjudicaciones = df_adjudicaciones.drop(columns = ['codigogrupo','descripcion_grupo','descripcion_familia'])

In [189]:
def items_nuevos_para_postor(codigoconvocatoria,fechaconvocatoria,df_convocatorias,df_adjudicaciones,df_postores, nivel_itemcubso = 'codigoclase', nivel_itemcubso_string = 'clase'):
    #0. Identifico postores para esta convocatoria
    df_postores = df_postores.loc[df_postores['codigo_convocatoria'] == codigoconvocatoria]
    df_postores = df_postores[df_postores['ruc_detallepostor'] != '99999999999']
    postores_de_convocatoria = df_postores['ruc_detallepostor'].unique()
    postores_de_convocatoria_razonsocial = df_postores['detalle_postor'].unique()
    
    #1. Selecciono items de la convocatoria bajo analisis
    df_convocatorias = df_convocatorias.loc[df_convocatorias['codigoconvocatoria'] == codigoconvocatoria]
    df_convocatorias = df_convocatorias.loc[pd.notnull(df_convocatorias[nivel_itemcubso])]
    df_convocatorias = df_convocatorias.loc[df_convocatorias[nivel_itemcubso] != '<NA>-<NA>']
    items_convocatoria = df_convocatorias[nivel_itemcubso].unique()
    
    #2. Selecciono las adjudicaciones hechas previas a la fecha de convocatoria
    df_adjudicaciones = df_adjudicaciones.loc[(df_adjudicaciones['fecha_consentimiento_bp'] < fechaconvocatoria)]
    
    #3. Genero diccionario vacio para llenar postores e items nuevos
    diccionario = {}    
    if len(items_convocatoria)!=0:
        for postor,postor_razonsocial in zip(postores_de_convocatoria,postores_de_convocatoria_razonsocial):
            # Agrego el postor al diccionario
            items_nuevos = []
            diccionario[(postor,postor_razonsocial)] = items_nuevos
            #diccionario[postor] = items_nuevos
            # Selecciono adjudicaciones previas de este postor
            df_adjudicaciones_postor = df_adjudicaciones.loc[df_adjudicaciones['ruc_codigo_ganador'] == postor]
            df_adjudicaciones_postor = df_adjudicaciones_postor.loc[pd.notnull(df_adjudicaciones_postor[nivel_itemcubso])]
            # Selecciono items de las adjudicaciones ganadas por este postor
            df_items_adjudicados = df_adjudicaciones_postor.loc[df_adjudicaciones_postor[nivel_itemcubso] != '<NA>-<NA>']
            items_adjudicados = list(df_items_adjudicados[nivel_itemcubso].unique())
            if len(items_adjudicados)==0:
                #diccionario[postor] =  'sin items adjudicados o dato de item'
                continue
            else: 
                # Agrego al diccionario los items del postor para los que nunca adjudico antes. 
                for item in items_convocatoria:
                    if item in items_adjudicados:
                        continue
                    else:
                        if df_convocatorias.loc[df_convocatorias[nivel_itemcubso] == item].size != 0:
                            item_string = df_convocatorias.loc[df_convocatorias[nivel_itemcubso] == item][nivel_itemcubso_string].unique()[0]
                        else: 
                            item_string = None
                        diccionario[(postor,postor_razonsocial)].append((item, item_string, items_adjudicados))
                        #diccionario[postor].append((item, item_string, items_adjudicados))
    #else:
        #diccionario['todos_los_rucs'] = 'sin items adjudicados o dato de item'

    # Limpio el diccionario cuando no hay items que mostrar                
    diccionario = {key:val for key, val in diccionario.items() if len(val) != 0}
    return diccionario

In [190]:
# Genero atributos de bandera 5a
df_bandera5a_2021['bandera5a_items_nunca_antes_adjudicados_por_postor'] = df_bandera5a_2021.apply(lambda x: items_nuevos_para_postor(x['codigoconvocatoria'],
                                                                               x['fechaconvocatoria'],
                                                                               convocatorias,
                                                                               df_adjudicaciones,
                                                                               postores_cc,
                                                                               nivel_itemcubso = 'grupo_familia',# codigo_grupo #'grupo_familia_clase',#'grupo_familia',#nivel_itemcubso = 'codigo_grupo',
                                                                               nivel_itemcubso_string = 'familia'),# grupo #'clase'), #nivel_itemcubso_string = 'grupo'),
                                           axis = 1)

<ipython-input-190-b6d8aea7f623>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera5a_2021['bandera5a_items_nunca_antes_adjudicados_por_postor'] = df_bandera5a_2021.apply(lambda x: items_nuevos_para_postor(x['codigoconvocatoria'],


In [191]:
# Reemplazar diccionarios vacios por null
df_bandera5a_2021['bandera5a_items_nunca_antes_adjudicados_por_postor'] = df_bandera5a_2021.apply(lambda x: None if not(x['bandera5a_items_nunca_antes_adjudicados_por_postor']) else x['bandera5a_items_nunca_antes_adjudicados_por_postor'], axis = 1)

<ipython-input-191-90b9ff99c9ee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera5a_2021['bandera5a_items_nunca_antes_adjudicados_por_postor'] = df_bandera5a_2021.apply(lambda x: None if not(x['bandera5a_items_nunca_antes_adjudicados_por_postor']) else x['bandera5a_items_nunca_antes_adjudicados_por_postor'], axis = 1)


In [192]:
# Bandera 5a
df_bandera5a_2021['bandera_5a'] = df_bandera5a_2021.apply(lambda x: 'No' if (x['bandera5a_items_nunca_antes_adjudicados_por_postor'] is None) or (pd.isnull(x['bandera5a_items_nunca_antes_adjudicados_por_postor'])) 
                                                                else 'Si', axis = 1)

<ipython-input-192-740269db974e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera5a_2021['bandera_5a'] = df_bandera5a_2021.apply(lambda x: 'No' if (x['bandera5a_items_nunca_antes_adjudicados_por_postor'] is None) or (pd.isnull(x['bandera5a_items_nunca_antes_adjudicados_por_postor']))


In [194]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('5a',start,end, tiempos)

### Bandera 8: Se ha adjudicado buena pro, a pesar de que antes hubo denuncia o dictamen que indica que se declare la nulidad del procedimiento


In [199]:
# Start time de bandera
start = datetime.datetime.now()

In [200]:
# Selecciono variables relevantes para la bandera
df_bandera8 = df_agg_cac[['codigoconvocatoria']]

In [201]:
# DGR
df_bandera8_aso = dgr_aso[['codigoconvocatoria','disposicion','fecha_oficio']].rename(columns = {'disposicion':'resultado','fecha_oficio':'fecha'})
df_bandera8_dict = dgr_dict[['codigoconvocatoria','conclusion_final','fecha_emision']].rename(columns = {'conclusion_final':'resultado','fecha_emision':'fecha'})
df_bandera8_dgr = pd.concat([df_bandera8_aso,df_bandera8_dict])

In [202]:
# Adjudicaciones solo items contratados, adjudicados, consentidos o apelados
df_bandera8_adj = adjudicaciones.loc[adjudicaciones['estado_item'].isin(['Contratado', 'Adjudicado', 'Consentido', 'Apelado'])][['codigoconvocatoria','fecha_consentimiento_bp','n_item','codigoitem','itemcubso']]

In [203]:
# Join adjudicaciones con DGR ASO
df_bandera8_adj_dgr = df_bandera8_adj.join(df_bandera8_dgr.set_index(['codigoconvocatoria']),
                                                         on = ['codigoconvocatoria'],
                                                         how = 'left',
                                                         rsuffix = '_dgr')

In [204]:
# Nan a None
df_bandera8_adj_dgr = df_bandera8_adj_dgr.where(pd.notnull(df_bandera8_adj_dgr),None)

In [205]:
# Calculo diferencia entre fecha buena pro y oficio para cada idem
df_bandera8_adj_dgr['bandera_8_diferencia_fechabuenapro_fecha'] =  df_bandera8_adj_dgr['fecha'] - df_bandera8_adj_dgr['fecha_consentimiento_bp']

In [206]:
# Genero Bandera a nivel item
df_bandera8_adj_dgr['bandera_8'] = df_bandera8_adj_dgr.apply(lambda x: 'Si' if (x['bandera_8_diferencia_fechabuenapro_fecha'] < pd.Timedelta(0)) & (x['resultado'] in (['NULIDAD','Fundado'])) else 'No', axis = 1)



In [207]:
# Me quedo solo con los items que tienen bandera = si
df_bandera8_adj_dgr = df_bandera8_adj_dgr.loc[df_bandera8_adj_dgr['bandera_8'] == 'Si']

In [208]:
# Genero atributos 
df_bandera8_adj_dgr['bandera_8_atributos'] = ' N_item:' + df_bandera8_adj_dgr['n_item'].astype(str) + ';' +\
                                             ' Codigoitem:' + df_bandera8_adj_dgr['codigoitem'].astype(str) + ';' +\
                                             ' Itemcubso:' + df_bandera8_adj_dgr['itemcubso'].astype(str) + ';' +\
                                             ' Fecha Consentimiento BP:' + df_bandera8_adj_dgr['fecha_consentimiento_bp'].astype(str) + ';' +\
                                             ' Fecha:' + df_bandera8_adj_dgr['fecha'].astype(str) + ';' +\
                                             ' Resultado:' + df_bandera8_adj_dgr['resultado'].astype(str) + ',' 

In [209]:
# Flatten a nivel codigo convocatoria
df_bandera8_adj_dgr = pd.DataFrame(df_bandera8_adj_dgr.groupby('codigoconvocatoria', as_index = False)['bandera_8_atributos'].apply(' | '.join))

In [210]:
# Join con df_bandera_8 
df_bandera8 = df_bandera8.join(df_bandera8_adj_dgr.set_index('codigoconvocatoria'),
                               on = 'codigoconvocatoria',
                               how = 'left')

In [211]:
# Nan a None
df_bandera8 = df_bandera8.where(pd.notnull(df_bandera8),None)

In [212]:
# Bandera a nivel cod convocatoria
df_bandera8['bandera_8'] = df_bandera8.apply(lambda x: 'No' if x['bandera_8_atributos'] is None else 'Si', axis = 1)

In [213]:
# Renombro columna
df_bandera8 = df_bandera8.rename(columns = {'bandera_8_atributos':'bandera8_FechaAdj_Posterior_FechaDGR'})

In [218]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('8',start,end, tiempos)

### Bandera 13: Se ha hecho la adjudicación a un ganador no habido no hallado.

In [219]:
# Start time de bandera
start = datetime.datetime.now()

In [220]:
# Selecciono variables relevantes para la bandera
df_bandera13 = df_agg_cac[['codigoconvocatoria','anio']]

In [221]:
# Genero copia de tabla adjudicaciones
Bandera13_adjudicaciones = adjudicaciones.copy()

In [222]:
# Me quedo con un codigoconvocatoria-ruc por adjudicacion
Bandera13_adjudicaciones = Bandera13_adjudicaciones[['codigoconvocatoria','ruc_codigo_ganador','ganador','tipo_proveedor']].drop_duplicates()

In [223]:
# Ruc de tabla SUNAT como string para poder hacer el join
sunat['ruc'] = sunat['ruc'].astype(str)

In [224]:
# Join de bandera13_adjudicaciones y sunat
Bandera13_adjudicaciones = Bandera13_adjudicaciones.join(sunat[['ruc','eshabido']].set_index('ruc'),
                                                        on = 'ruc_codigo_ganador',
                                                        how = 'left')

In [225]:
# Transformo Nan a 'No encontrado en Sunat'
Bandera13_adjudicaciones = Bandera13_adjudicaciones.where(pd.notnull(Bandera13_adjudicaciones),'No encontrado en Sunat')

In [226]:
# Bandera 13 a nivel ruc-convocatoria
Bandera13_adjudicaciones['bandera_13'] = Bandera13_adjudicaciones.apply(lambda x: 'No aplica' if  'No encontrado en Sunat' in x['eshabido']
                                                                        else ('Si' if x['eshabido'] in ('NO HABIDO','NO HALLADO') else 'No'), axis = 1)

In [227]:
# Me quedo solamente con las conv que tienen al menos un ruc no habido/no hallado o 'no aplica'
Bandera13_adjudicaciones_si_noaplica = Bandera13_adjudicaciones.loc[Bandera13_adjudicaciones['bandera_13'].isin(['Si','No aplica'])]

In [228]:
# Todas las columnas a concatenar como strings
Bandera13_adjudicaciones_si_noaplica ['ruc_codigo_ganador'] = Bandera13_adjudicaciones_si_noaplica ['ruc_codigo_ganador'].astype(str)
Bandera13_adjudicaciones_si_noaplica ['ganador'] = Bandera13_adjudicaciones_si_noaplica ['ganador'].astype(str)
Bandera13_adjudicaciones_si_noaplica ['eshabido'] = Bandera13_adjudicaciones_si_noaplica['eshabido'].astype(str)
Bandera13_adjudicaciones_si_noaplica ['tipo_proveedor'] = Bandera13_adjudicaciones_si_noaplica['tipo_proveedor'].astype(str)

<ipython-input-228-fd2f6b4eabb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandera13_adjudicaciones_si_noaplica ['ruc_codigo_ganador'] = Bandera13_adjudicaciones_si_noaplica ['ruc_codigo_ganador'].astype(str)
<ipython-input-228-fd2f6b4eabb7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandera13_adjudicaciones_si_noaplica ['ganador'] = Bandera13_adjudicaciones_si_noaplica ['ganador'].astype(str)
<ipython-input-228-fd2f6b4eabb7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [229]:
# Concateno columnas para armar atributos de bandera 13
Bandera13_adjudicaciones_si_noaplica['bandera_13_atributos'] = 'Ruc:' + Bandera13_adjudicaciones_si_noaplica ['ruc_codigo_ganador'] +';'+ ' Razon social:' + Bandera13_adjudicaciones_si_noaplica ['ganador'] +';'+ ' Tipo:' + Bandera13_adjudicaciones_si_noaplica ['tipo_proveedor']  +';'+ ' Estado:' + Bandera13_adjudicaciones_si_noaplica ['eshabido'] + ' '
#Bandera13_adjudicaciones_si_noaplica['bandera_13_atributos'] = 'Ruc:' + Bandera13_adjudicaciones_si_noaplica ['ruc_codigo_ganador'] +';'+ ' Tipo:' + Bandera13_adjudicaciones_si_noaplica ['tipo_proveedor']  +';'+ ' Estado:' + Bandera13_adjudicaciones_si_noaplica ['eshabido'] + ' '

<ipython-input-229-5ca74cd8e3ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandera13_adjudicaciones_si_noaplica['bandera_13_atributos'] = 'Ruc:' + Bandera13_adjudicaciones_si_noaplica ['ruc_codigo_ganador'] +';'+ ' Razon social:' + Bandera13_adjudicaciones_si_noaplica ['ganador'] +';'+ ' Tipo:' + Bandera13_adjudicaciones_si_noaplica ['tipo_proveedor']  +';'+ ' Estado:' + Bandera13_adjudicaciones_si_noaplica ['eshabido'] + ' '


In [230]:
# Flatten del df para que quede a nivel codigoconvocatoria y haga un string con todos los items con bandera22
Bandera13_adjudicaciones_si_noaplica  = pd.DataFrame(Bandera13_adjudicaciones_si_noaplica .groupby(['codigoconvocatoria'], as_index = False)['bandera_13_atributos'].apply(';'.join))

In [231]:
# Join df_bandera_13 con Bandera13_adjudicaciones_si_noaplica
df_bandera13 = df_bandera13.join(Bandera13_adjudicaciones_si_noaplica.set_index('codigoconvocatoria'), 
                                on = 'codigoconvocatoria',
                                how ='left')

In [232]:
# Cambio Nan por Null
df_bandera13 = df_bandera13.where(pd.notnull(df_bandera13),None)

In [233]:
# Genero bandera 13 a nivel codigo convocatoria
estados = ['NO HABIDO','NO HALLADO']
df_bandera13['bandera_13'] = df_bandera13.apply(lambda x: 'No' if x['bandera_13_atributos'] is None 
                                                                                       else ('Si' if any( i in x['bandera_13_atributos'] for i in estados) else 'No'), 
                                                                        axis = 1)

In [234]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('13',start,end, tiempos)

In [235]:
df_bandera13 = df_bandera13.rename(columns = {'bandera_13_atributos':'bandera_13_RUC_nohabido_nohallado'})

### Bandera 15: El monto del postor es demasiado cercano al monto referencial

In [239]:
# Start time de bandera
start = datetime.datetime.now()

In [240]:
# Selecciono variables relevantes para la bandera
df_bandera15 = df_agg_cac[['codigoconvocatoria','anio','fechaconvocatoria','objetocontractual','tipoprocesoseleccion']]

In [241]:
# Genero dataframe de convocatorias a nivel conv - item
convocatorias_items = convocatorias.drop_duplicates(['codigoconvocatoria','n_item','codigoitem','itemcubso','estadoitem'])
postores_items = postores.drop_duplicates(['codigo_convocatoria','n_item','ruc_detallepostor','detalle_postor'])

In [242]:
# Seleccionamos items cuyo estado es 'Contratado','Consentido', 'Adjudicado','Convocado'
postores_items = postores_items.loc[postores_items['estado_item'].isin(['Contratado','Convocado', 'Consentido','Adjudicado'])]
convocatorias_items = convocatorias_items.loc[convocatorias_items['estadoitem'].isin(['Contratado','Convocado', 'Consentido','Adjudicado'])]

In [243]:
# Join de convocatorias_item con postores_item
conv_postores = convocatorias_items[['codigoconvocatoria','n_item','monto_referencial_item','codigoitem','itemcubso']]\
                .join(postores_items[['codigo_convocatoria','n_item','ruc_detallepostor','detalle_postor','monto']].set_index(['codigo_convocatoria','n_item']), 
                  on = ['codigoconvocatoria','n_item'],
                  how = 'left')

In [244]:
# Genero dataframe que cuenta cantidad de postores por convocatoria-item:
conv_postores_q_postores = conv_postores.groupby(['codigoconvocatoria','n_item'], as_index = False).count()[['codigoconvocatoria','n_item','monto_referencial_item']]
conv_postores_q_postores.rename(columns = {'monto_referencial_item': 'cantidad_postores'}, inplace = True)

In [245]:
# Join conv_postores con coluna que indica cantidad de postores para conv-item
conv_postores = conv_postores.join(conv_postores_q_postores.set_index(['codigoconvocatoria','n_item']),
                                  on = ['codigoconvocatoria','n_item'],
                                 how = 'left')

In [246]:
# Me quedo solo con los que tienen un unico postor
conv_postores = conv_postores.loc[conv_postores['cantidad_postores'] == 1]

In [247]:
# Calculo ratio precio del postor/precio de referencia
conv_postores['bandera15_ratio_montorefitem_montopostoritem'] = (conv_postores['monto'] * 1.00 / conv_postores['monto_referencial_item'] * 1.00 )* 100

In [248]:
# Genero bandera a nivel convocatoria-item
conv_postores['bandera15'] = conv_postores.apply(lambda x: 'Si' if (x['bandera15_ratio_montorefitem_montopostoritem'] > 95) & (x['bandera15_ratio_montorefitem_montopostoritem'] < 105) else ' No', axis = 1)
                                                 

In [249]:
# Selecciono solo las conv-item con bandera = Si
conv_postores_si = conv_postores.loc[conv_postores['bandera15'] == 'Si']

In [250]:
# Genero columna que resume info del item, monto ref item y monto del postor
conv_postores_si['bandera15_attr'] = 'N_item: ' + \
                                    conv_postores_si['n_item'].astype(int).astype(str) +\
                                    ';Codigoitem: ' + \
                                    conv_postores_si['codigoitem'].astype(str) +\
                                    ';Itemcubso: ' + \
                                    conv_postores_si['itemcubso'].astype(str) +\
                                    ';Monto_Ref_Item: ' + \
                                    conv_postores_si['monto_referencial_item'].astype(int).astype(str) +\
                                    ';Monto_Postor_Item: ' + \
                                    conv_postores_si['monto'].astype(int).astype(str) 

<ipython-input-250-d912fec0fca7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conv_postores_si['bandera15_attr'] = 'N_item: ' + \


In [251]:
# Agrupo a nivel conv-postor y genero una columna con la bandera15_attr dentro de una lista
conv_postores_si_agg = conv_postores_si.groupby(['codigoconvocatoria','ruc_detallepostor','detalle_postor'])['bandera15_attr'].apply(list).reset_index(name='bandera15_atributos')

In [252]:
# Genero nueva columna que a la info resumen agregra el ruc del postor
#conv_postores_si_agg['bandera15_items_cercanos_valor_referencia'] = '{Postor: ' + conv_postores_si_agg['ruc_detallepostor'] +','+ conv_postores_si_agg['detalle_postor'] + ':' + conv_postores_si_agg['bandera15_atributos'].astype(str) + '}'
conv_postores_si_agg['bandera15_items_cercanos_valor_referencia'] = '{Postor: ' + conv_postores_si_agg['ruc_detallepostor'] +':' + conv_postores_si_agg['bandera15_atributos'].astype(str) + '}'

In [253]:
# Flatten de la tabla para que todo quede a nivel convocatoria
conv_postores_si_agg = conv_postores_si_agg.groupby(['codigoconvocatoria'])['bandera15_items_cercanos_valor_referencia'].apply(list).reset_index(name='bandera15_items_cercanos_valor_referencia')

In [254]:
df_bandera15 = df_bandera15.join(conv_postores_si_agg.set_index('codigoconvocatoria'), 
                                 on = 'codigoconvocatoria', how = 'left')

In [255]:
def bandera15(atributos,objetocontractual,tipoprocesoseleccion):
    if tipoprocesoseleccion == 'Contratación Directa':
        resultado = 'No aplica'
    else:
        if objetocontractual in ('Servicio','Bien'):
            if isinstance(atributos, list):
                resultado = 'Si'
            else:
                resultado = 'No'
        else:
            resultado = 'No aplica'
    return (resultado)

In [256]:
df_bandera15['bandera15'] = df_bandera15.apply(lambda x: bandera15(x['bandera15_items_cercanos_valor_referencia'],
                                                                   x['objetocontractual'],
                                                                  x['tipoprocesoseleccion']), 
                                               axis = 1)

In [257]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('15',start,end, tiempos)

### Bandera 16: A una convocatoria se presentó un postor que consistentemente gana contratos con este ente convocante


In [269]:
# Start time de bandera
start = datetime.datetime.now()

In [270]:
# Selecciono variables para la bandera
df_bandera16 = df_agg_cac[['anio','codigoconvocatoria','ruc_entidad','entidad']]
#df_bandera16 = df_bandera16.loc[df_bandera16['anio'] == '2021']

In [271]:
# Genero un dataframe de convocatorias 2018-2020 con columnas que me interesa (saco duplicados)
df_bandera16_conv = convocatorias[['codigoconvocatoria','anio','ruc_entidad','entidad','n_item','grupo_familia','familia']].drop_duplicates()
df_bandera16_conv = df_bandera16_conv.loc[df_bandera16_conv['anio'].isin([2018,2019,2020])]

In [272]:
# Genero un dataframe de postores por conv y n_item saco duplicados
df_bandera16_postores = postores[['codigo_convocatoria','n_item','ruc_detallepostor','detalle_postor']].drop_duplicates()

In [273]:
# Tabla desagregada de convocatorias y sus postores periodo 2018-2020
df_bandera16_conv_postores = df_bandera16_conv.join(df_bandera16_postores.set_index(['codigo_convocatoria','n_item']),
                                                on = (['codigoconvocatoria','n_item']),
                                                how = 'inner')

In [274]:
# Tabla agregada de convocatorias y sus postores periodo 2018-2020
df_bandera16_conv_postores_agg = df_bandera16_conv_postores\
.groupby(['ruc_detallepostor','ruc_entidad','grupo_familia','familia'], as_index = False).count()\
[['ruc_detallepostor','ruc_entidad','grupo_familia','familia','codigoconvocatoria']]\
.rename(columns = {'codigoconvocatoria': 'cantidad_postulaciones'})

In [275]:
# Tabla desagregada de adjudicaciones
df_bandera16_adj = adjudicaciones[['codigoconvocatoria','ruc_entidad','ruc_codigo_ganador','ganador','grupo_familia','descripcion_familia','anio']].drop_duplicates()
df_bandera16_adj = df_bandera16_adj.loc[df_bandera16_adj['anio'].isin([2018,2019,2020])]

In [276]:
# Tabla agregada de adjudicaciones
df_bandera16_adj_agg = df_bandera16_adj.groupby(['ruc_codigo_ganador','ganador','ruc_entidad','grupo_familia'], as_index = False)\
.count() [['ruc_codigo_ganador','ganador','ruc_entidad','grupo_familia','codigoconvocatoria']]\
.rename(columns = {'codigoconvocatoria': 'cantidad_adjudicadas'})

In [277]:
# Join tabla agregada de cantidad de postulaciones y tabla agregada de cantidad adjudicadas
df_bandera16_agg = df_bandera16_conv_postores_agg.join(df_bandera16_adj_agg.set_index(['ruc_codigo_ganador','ruc_entidad','grupo_familia']),
                                                     on = (['ruc_detallepostor','ruc_entidad','grupo_familia']),
                                                    how = 'left')

In [278]:
# Reemplazo Nan por 0
df_bandera16_agg = df_bandera16_agg.where(pd.notnull(df_bandera16_agg),0)

In [279]:
# Calculo ratio cantidad_adjudicada / cantidad_postulacion para cada postor-entidad-familia
df_bandera16_agg['bandera_16_ratio_cantadjudicaciones_cantpostulaciones'] = df_bandera16_agg['cantidad_adjudicadas'] * 1.00 / df_bandera16_agg['cantidad_postulaciones'] * 1.00 * 100 

In [280]:
# Genero bandera a nivel postor-entidad-familia - 90% (sin contar los casos 1:1)
df_bandera16_agg['bandera_16'] =  df_bandera16_agg.apply(lambda x: 'Si' 
                                                        if (x['bandera_16_ratio_cantadjudicaciones_cantpostulaciones'] > 90) & (x['cantidad_adjudicadas']!= 1 and x['cantidad_postulaciones'] != 1)
                                                         else 'No', axis = 1)

In [281]:
# Genero ID postor-entidad-familia
#df_bandera16_agg['id_entidad_familia'] = df_bandera16_agg['ruc_entidad'] + '-' + df_bandera16_agg['codigo_familia']
df_bandera16_agg['id_postor_entidad_familia'] = df_bandera16_agg['ruc_detallepostor'].astype(str) + '-' + df_bandera16_agg['ruc_entidad'].astype(str) + '-' + df_bandera16_agg['grupo_familia'].astype(str)

In [282]:
df_bandera16_agg = df_bandera16_agg[['id_postor_entidad_familia','cantidad_postulaciones','cantidad_adjudicadas','bandera_16_ratio_cantadjudicaciones_cantpostulaciones','bandera_16']]

In [283]:
# Genero tabla de postores para convocatorias
df_postores_cc = postores[['codigo_convocatoria','ruc_detallepostor','detalle_postor']].drop_duplicates()

In [284]:
# Me quedo con convocatorias 
df_convocatorias = convocatorias[['codigoconvocatoria','ruc_entidad','entidad','grupo_familia','familia']].drop_duplicates()
df_convocatorias['id_entidad_familia'] = convocatorias['ruc_entidad'].astype(str) + '-' + convocatorias['grupo_familia'].astype(str)

In [285]:
# Join convocatorias y sus postores
df_convocatorias = df_convocatorias.join(df_postores_cc.set_index('codigo_convocatoria'), on ='codigoconvocatoria',
                                        how = 'left')

In [286]:
# Genero ID
df_convocatorias['id_postor_entidad_familia'] = df_convocatorias['ruc_detallepostor'] + '-' + df_convocatorias['id_entidad_familia']

In [287]:
df_convocatorias = df_convocatorias.join(df_bandera16_agg.set_index('id_postor_entidad_familia'),
                                         on = 'id_postor_entidad_familia',
                                         how = 'left',
                                         rsuffix = '_agg')

In [288]:
# Me quedo con los que bandera = si
df_convocatorias = df_convocatorias.loc[df_convocatorias['bandera_16'] == 'Si']

In [289]:
df_convocatorias['bandera_16_atributos'] = 'Ruc postor:' + df_convocatorias['ruc_detallepostor'] +\
                                           '-Razon social:' + df_convocatorias['detalle_postor'].astype(str) +\
                                           '-CodigoFamilia:' + df_convocatorias['grupo_familia'].astype(str) +\
                                           '-Familia:' + df_convocatorias['familia'] +\
                                           '-Cantidad postulaciones:' + df_convocatorias['cantidad_postulaciones'].astype(int).astype(str) +\
                                           '-Cantidad adjudicadas:' + df_convocatorias['cantidad_adjudicadas'].astype(int).astype(str) +\
                                           '-Porcentaje adjudicadas/postulaciones:' + df_convocatorias['bandera_16_ratio_cantadjudicaciones_cantpostulaciones'].astype(int).astype(str)+'%' 

In [290]:
df_convocatorias = pd.DataFrame(df_convocatorias.groupby('codigoconvocatoria',as_index = False)['bandera_16_atributos'].apply(';'.join))

In [291]:
# Join con df_banderas16 con df_convocatorias
df_bandera16 = df_bandera16.join(df_convocatorias.set_index('codigoconvocatoria'),
                                on = 'codigoconvocatoria',
                                how = 'left')

In [292]:
df_bandera16 = df_bandera16.where(pd.notnull(df_bandera16),None)

In [293]:
# Genero bandera
df_bandera16['bandera_16'] = df_bandera16.apply(lambda x: 'No' if x['bandera_16_atributos'] is None else 'Si', axis = 1)

In [294]:
# Rename columns
df_bandera16 = df_bandera16.rename(columns ={'ruc_entidad' : 'bandera_16_rucentidad',
                                             'entidad': 'bandera_16_entidad',
                                            'bandera_16_atributos': 'bandera_16_postores_codflia_porcentaje_adj_vs_postulado'})

In [295]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('16',start,end, tiempos)

### Bandera 17: El tiempo transcurrido entre buena pro y consentimiento de buena pro es inferior al límite marcado por la legislación

In [308]:
# Start time de bandera
start = datetime.datetime.now()

In [309]:
# Selecciono columnas relevantes para la bandera a construir
df_bandera_17 = df_agg_cac[['anio','codigoconvocatoria','tipoprocesoseleccion']]

In [310]:
# Calculo diferencia de dias entre fechas
Bandera17_adjudicaciones = adjudicaciones.copy()
Bandera17_adjudicaciones['fecha_buenapro'] = Bandera17_adjudicaciones['fecha_buenapro'].fillna('1900-01-01') # nan a 1900 para poder computar np.busday
Bandera17_adjudicaciones['fecha_consentimiento_bp'] = Bandera17_adjudicaciones['fecha_consentimiento_bp'].fillna('1900-01-01') # nan a 1900 para poder computar np.busday

# Calculo diferencia de dias entre fechas excluyendo fin de semanas y feriados
Bandera17_adjudicaciones['bandera_17_dif_buenapro_consentimiento_bp'] = np.busday_count(Bandera17_adjudicaciones['fecha_buenapro'].values.astype('datetime64[D]'),
                               Bandera17_adjudicaciones['fecha_consentimiento_bp'].values.astype('datetime64[D]'), 
                              holidays = fecha_feriados)

# Asigno NA a los casos en los que fecha convocatoria y/o fechapresentacionpropuesta es null
Bandera17_adjudicaciones['bandera_17_dif_buenapro_consentimiento_bp'] = Bandera17_adjudicaciones.apply(lambda x: 'None' if (x['fecha_buenapro'] == '1900-01-01') or (x['fecha_consentimiento_bp'] == '1900-01-01 00:00:00')
                                                                                      else x['bandera_17_dif_buenapro_consentimiento_bp'], axis = 1)

In [311]:
Bandera17_adjudicaciones['consorcio'] = Bandera17_adjudicaciones.apply(lambda x: x['ruc_codigo_ganador'] 
                                                                       if x['tipo_proveedor'] == 'Consorcio' else 'No',
                                                                      axis = 1)

In [312]:
Bandera17_adjudicaciones['ruc_proveedor'] = Bandera17_adjudicaciones.apply(lambda x: x['ruc_miembro'] 
                                                                       if x['tipo_proveedor'] == 'Consorcio' else x['ruc_codigo_ganador'],
                                                                      axis = 1)

In [313]:
# Defino cantidad de dias de acuerdo a tipo de proceso
def bandera_17_umbral_dias(tipoprocesoseleccion):
    """
    Determina cantidad de dias máximos que pueden transcurrir entre fecha de buena pro y fecha de consentimiento de buena pro
    de acuerdo al tipo de proceso de selección
    
    Params
    ------
    tipoprocesoseleccion(string): Tipo de proceso de seleccion de la convocatoria, por ejemplo 'Adjudicación Simplificada'
    
    Returns
    -------
    Cantidad de días máximos que pueden transcurrir entre fecha de buena pro y fecha de consentimiento de la buena pro
    dado el tipo de proceso de selección
    
    """
    if tipoprocesoseleccion == 'Concurso Público':
        resultado = 9
    elif tipoprocesoseleccion == 'Licitación Pública':
        resultado = 9
    elif tipoprocesoseleccion == 'Contratación Directa':
        resultado = None
    elif tipoprocesoseleccion == 'Adjudicación Simplificada':
        resultado = 6
    elif tipoprocesoseleccion == 'Subasta Inversa Electrónica':
        resultado = 6
    elif tipoprocesoseleccion == 'Comparación de Precios':
        resultado = 6
    else:
        resultado = None
    return(resultado)

In [314]:
# Agrego columna que indica umbral
Bandera17_adjudicaciones['bandera_17_umbral'] = Bandera17_adjudicaciones.apply(lambda x: bandera_17_umbral_dias(x['tipoprocesoseleccion']), axis = 1)

In [315]:
# Computo bandera 17 a nivel item
Bandera17_adjudicaciones['bandera_17'] = Bandera17_adjudicaciones.apply(lambda x: 'Si' 
                                               if (x['bandera_17_dif_buenapro_consentimiento_bp'] < x['bandera_17_umbral']) and (x['bandera_17_dif_buenapro_consentimiento_bp'] >= 0)  
                                               else 'No', 
                                               axis = 1)

In [316]:
# Selecciono columnas relevantes
Bandera17_adjudicaciones = Bandera17_adjudicaciones[['codigoconvocatoria','tipoprocesoseleccion','ruc_codigo_ganador','ganador','codigoitem','fecha_buenapro','fecha_consentimiento_bp','bandera_17_dif_buenapro_consentimiento_bp','bandera_17_umbral','bandera_17','ruc_proveedor','consorcio']]

In [317]:
# Me quedo unicamente con convocatorias que tengan al menos un item con bandera 17
Bandera17_adjudicaciones = Bandera17_adjudicaciones.loc[Bandera17_adjudicaciones['bandera_17'] == 'Si']

In [318]:
# Me quedo con convocatorias que hayan tenido solamente un postor

# Genero tabla con codigoconvocatoria y postulantes unicos 
# (en tabla original esta aperturado por item y duplica postor)
postores_cc = postores.groupby(['codigo_convocatoria','ruc_detallepostor','detalle_postor'], as_index =  False).count()
convocatoria_multiples_postores = postores_cc.groupby('codigo_convocatoria',as_index = False).count()[['codigo_convocatoria','ruc_detallepostor','detalle_postor']]
convocatoria_multiples_postores = convocatoria_multiples_postores.rename(columns = {'ruc_detallepostor': 'cantidad_postores'})

# Selecciono convocatorias con mas de un postor
convocatoria_multiples_postores = convocatoria_multiples_postores.loc[convocatoria_multiples_postores['cantidad_postores']> 1]

# Join con tabla Bandera17_adjudicaciones
Bandera17_adjudicaciones = Bandera17_adjudicaciones.join(convocatoria_multiples_postores.set_index('codigo_convocatoria'),
                                                        on = 'codigoconvocatoria',
                                                        how = 'inner')


In [320]:
# Cambio los tipos de datos de columnas que usare como atributos a string
Bandera17_adjudicaciones['consorcio'] = Bandera17_adjudicaciones['consorcio'].astype(str)
Bandera17_adjudicaciones['ruc_proveedor'] = Bandera17_adjudicaciones['ruc_proveedor'].astype(str)
Bandera17_adjudicaciones['ganador'] = Bandera17_adjudicaciones['ganador'].astype(str)
Bandera17_adjudicaciones['codigoitem'] = Bandera17_adjudicaciones['codigoitem'].astype(str)
Bandera17_adjudicaciones['fecha_buenapro'] = Bandera17_adjudicaciones['fecha_buenapro'].astype(str)
Bandera17_adjudicaciones['fecha_consentimiento_bp'] = Bandera17_adjudicaciones['fecha_consentimiento_bp'].astype(str)
Bandera17_adjudicaciones['bandera_17_dif_buenapro_consentimiento_bp'] = Bandera17_adjudicaciones['bandera_17_dif_buenapro_consentimiento_bp'].astype(str)

In [321]:
# Genero una columna que tendra todos los atributos que quiero - concatenando las columnas

Bandera17_adjudicaciones['bandera17_atributos'] = 'Consorcio:' + Bandera17_adjudicaciones['consorcio'] + ';' +\
' Ruc o Ruc miembro consorcio:' + Bandera17_adjudicaciones['ruc_proveedor'] + ';' +\
' Razon social:' + Bandera17_adjudicaciones['ganador'] + ';' +\
' CodigoItem:' + Bandera17_adjudicaciones['codigoitem'] + ';' + \
' Fecha bp: ' + Bandera17_adjudicaciones['fecha_buenapro'] + ';' +\
' Fecha consentimiento bp: ' + Bandera17_adjudicaciones['fecha_consentimiento_bp'] + ';' +\
' Diferencia:' + Bandera17_adjudicaciones['bandera_17_dif_buenapro_consentimiento_bp']

In [322]:
# Flatten del df para que quede a nivel codigoconvocatoria y haga un string con todos los items con bandera17
Bandera17_adjudicaciones = pd.DataFrame(Bandera17_adjudicaciones.groupby(['codigoconvocatoria'], as_index = False)['bandera17_atributos'].apply('|'.join))

In [323]:
# Agrego columna que indica umbral
df_bandera_17['bandera_17_umbral'] = df_bandera_17.apply(lambda x: bandera_17_umbral_dias(x['tipoprocesoseleccion']), axis = 1)

<ipython-input-323-46bbce69c349>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandera_17['bandera_17_umbral'] = df_bandera_17.apply(lambda x: bandera_17_umbral_dias(x['tipoprocesoseleccion']), axis = 1)


In [324]:
# Join df_bandera_17 con Bandera17_adjudicaciones
df_bandera_17 = df_bandera_17.join(Bandera17_adjudicaciones.set_index('codigoconvocatoria'),
                                  on = 'codigoconvocatoria',
                                  how = 'left')

In [325]:
# Reemplazo Nan por None
df_bandera_17 = df_bandera_17.where(pd.notnull(df_bandera_17), None)

In [326]:
# Genero bandera a nivel convocaotira
df_bandera_17['bandera_17'] = df_bandera_17.apply(lambda x: 'No aplica' if x['tipoprocesoseleccion'] == 'Contratación Directa' else ('No' if x['bandera17_atributos'] is None 
                                                  else 'Si'), axis = 1)

In [327]:
# Renombro columna con atributo
df_bandera_17.rename(columns = {'bandera17_atributos':'bandera17_items_adjudicados_plazo_inferior_a_ley'}, inplace = True)

In [328]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('17',start,end, tiempos)

### Bandera 19: El monto adjudicado por unidad (precio unitario) es superior al valor histórico

In [336]:
# Start time de bandera
start = datetime.datetime.now()

In [337]:
# Selecciono variables relevantes para la bandera
df_bandera19 = df_agg_cac[['codigoconvocatoria','anio','fechaconvocatoria','objetocontractual','tipoprocesoseleccion']]

In [338]:
df_bandera19 = df_bandera19.loc[df_bandera19['anio'] == '2021']

(18363, 5)

In [339]:
# Solo bienes y solo convocatorias de 2021
df_bandera19_resto = df_bandera19.loc[df_bandera19['objetocontractual'] != 'Bien']
df_bandera19 = df_bandera19.loc[df_bandera19['objetocontractual'] == 'Bien']

In [340]:
# Genero precio unitario del item adjudicado
adjudicaciones['precio_unitario_adjudicado'] = round(adjudicaciones['monto_adjudicado_item']/adjudicaciones['cantidad_adjudicada_item'],2)

In [341]:
# Solo items contratados o adjudicados - Solo Bienes
df_adjudicaciones = adjudicaciones[adjudicaciones['estado_item'].isin(['Contratado','Adjudicado'])]
df_adjudicaciones = df_adjudicaciones.loc[df_adjudicaciones['objetocontractual'] == 'BIENES']
df_adjudicaciones = df_adjudicaciones[(df_adjudicaciones['moneda'].isin(['Soles','Nuevos Soles']))]

In [342]:
def precio_item_vs_historico(codigoconvocatoria, df_adjudicaciones, meses = 2):
    
    # Selecciono items de la convocatoria
    convocatoria_adjudicada = df_adjudicaciones.loc[df_adjudicaciones['codigoconvocatoria'] == codigoconvocatoria]
    items_adjudicados = convocatoria_adjudicada['codigoitem'].unique()
    
    diccionario = {}
    
    for item in items_adjudicados:
        if pd.isnull(item) == False: # Si el codigoitem existe  
            # Selecciono adjudicaciones previas de este tipo de item y misma unidad
            df_adj = df_adjudicaciones.loc[df_adjudicaciones['codigoitem'] == item]
            unidad_medida = convocatoria_adjudicada.loc[convocatoria_adjudicada['codigoitem'] == item]['unidad_medida'].values[0]
            df_adj = df_adj.loc[df_adj['unidad_medida'] == unidad_medida]
                    
            # Selecciono ventana temporal de analisis
            fecha_adjudicacion = pd.to_datetime(convocatoria_adjudicada[convocatoria_adjudicada['codigoitem'] == item]['fecha_consentimiento_bp'].values[0])
            fecha_inicio = (pd.to_datetime(fecha_adjudicacion) - pd.DateOffset(months=meses)).to_pydatetime()
            df_items = df_adj.loc[(df_adj['fecha_consentimiento_bp'] < fecha_adjudicacion) & (df_adj['fecha_consentimiento_bp']> fecha_inicio)] 
            
            if df_items.empty: 
                diccionario[f'item {item} -unidad {unidad_medida}'] = 'No existen adjudicaciones históricas en el periódo considerado para este item y unidad de medida'
            else:
                # Descarto outliers (algunos itemcubso no tienen las cantidades correctas e inflan precio unitario)
                outliers = df_items['precio_unitario_adjudicado'].describe([.95])
                df_items = df_items.loc[df_items['precio_unitario_adjudicado'] <= outliers['95%']]
                

                # Calculo media y SD de precio unitario para el item analizado
                Media = round(df_items['precio_unitario_adjudicado'].mean(),2)
                SD = round(df_items['precio_unitario_adjudicado'].std(),2)
                MasDosSD = round(Media + 2 * SD,2)                                  
                MenosDosSD = round(Media - 2 * SD,2)
                Convocatorias = df_items['codigoconvocatoria'].unique()
                
                if SD == 0:
                    cantidad_casos = df_items.shape[0]
                    diccionario[f'item {item}'] = 'No hay suficiente variabilidad de precio en el periodo considerado (Desvio estandar = 0 , Casos = cantidad_casos)'
                else:
                    # Miro adjudicaciones del item
                    adjudicaciones_item = convocatoria_adjudicada.loc[convocatoria_adjudicada['codigoitem'] == item]
                    precios_unitarios_adjudicados = adjudicaciones_item[['codigoitem','ruc_codigo_ganador','precio_unitario_adjudicado']]
                    
                    for index, row in precios_unitarios_adjudicados.iterrows(): #Puede haber mas de una adj para el mismo item y convocatoria
                        # Comparo el precio unitario del item adjudicado vs +/- 2 Desvios Estandar
                        if row['precio_unitario_adjudicado'] > MasDosSD:
                            diccionario[f'CodigoItem {item} -unidad {unidad_medida}'] = [f'precio unitario {row["precio_unitario_adjudicado"]}',f'precio media {Media}',f'precio std {SD}',f'umbral_superior {MasDosSD}', f'convocatorias: {Convocatorias}'] 
                        #elif row['precio_unitario_adjudicado'] < MenosDosSD:
                            #diccionario[f'CodigoItem {item} -unidad {unidad_medida}'] = [f'precio unitario {row["precio_unitario_adjudicado"]}',f'precio media {Media}',f'precio std {SD}',f'umbral_inferior {MenosDosSD}',f'umbral_superior {MasDosSD}', f'convocatorias: {Convocatorias}'] 
                        else:
                            continue
        else:
            diccionario[f'item {item}'] = 'Codigo de item es nulo'
            continue      
    return(diccionario)

In [343]:
# Generacion atributo
df_bandera19['bandera19_precio_unitario_item_vs_historico'] = df_bandera19.apply(lambda x: precio_item_vs_historico(x['codigoconvocatoria'],
                                                                                                                    df_adjudicaciones, 
                                                                                                                     meses = 2),
                                                                                axis = 1)

In [344]:
# Si diccionario esta vacio reemplazo por null
df_bandera19['bandera19_precio_unitario_item_vs_historico'] = df_bandera19.apply(lambda x: None if bool(x['bandera19_precio_unitario_item_vs_historico'])== False else x['bandera19_precio_unitario_item_vs_historico'] ,axis = 1) 

In [345]:
def generacion_bandera_19(atributo):
    if atributo is None:
        resultado = 'No'
    else:
        atributo = str(atributo.values())
        if 'precio unitario' in atributo:
            resultado = 'Si'
        elif 'Codigo de item es nulo' in atributo:
            resultado = 'No aplica'
        elif 'No existen adjudicaciones históricas' in atributo:
            resultado = 'No aplica'
        elif 'No hay suficiente variabilidad' in atributo:
            resultado = 'No aplica'
        else:
            resultado = None
    return(resultado)

In [346]:
# Generacion bandera
df_bandera19['bandera19'] = df_bandera19.apply(lambda x: 'No aplica' if x['objetocontractual'] != 'Bien' else
                                               generacion_bandera_19(x['bandera19_precio_unitario_item_vs_historico']),
                                              axis = 1)

In [347]:
# Agrego 'No aplica' de servicios, obras etc
df_bandera19_resto['bandera19'] = 'No aplica'
df_bandera19_resto['bandera19_precio_unitario_item_vs_historico'] = None

df_bandera19 = pd.concat([df_bandera19,df_bandera19_resto])

In [348]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('19',start,end, tiempos)

### Bandera 21: Adjudicación con el tipo de proceso de selección diferente al planificado

In [356]:
# Start time de bandera
start = datetime.datetime.now()

In [357]:
# Selecciono variables relevantes para la bandera
df_bandera_21 = df_agg_cac[['codigoconvocatoria','tipoprocesoseleccion']]

In [358]:
# Traigo proceso de seleccion de PAC y Adj
df_bandera_21 = df_bandera_21.join(df_agg_pcac[['codigoconvocatoria','tipoprocesoseleccion']]\
                                   .set_index('codigoconvocatoria'),
                                  on = 'codigoconvocatoria',
                                  how = 'left',
                                  rsuffix = '_pac')\
                              .join(adjudicaciones_agg[['codigoconvocatoria','tipoprocesoseleccion']]\
                                   .set_index('codigoconvocatoria'),
                                  on = 'codigoconvocatoria',
                                  how = 'left',
                                  rsuffix = '_adj')


In [359]:
# Nan a None
df_bandera_21 = df_bandera_21.where(pd.notnull(df_bandera_21),None)

In [360]:
df_bandera_21['bandera_21'] = df_bandera_21.apply(lambda x:'No aplica' if x['tipoprocesoseleccion_pac'] is None else 
                                                  ('No' if (x['tipoprocesoseleccion_pac'] == x['tipoprocesoseleccion_adj']) else 'Si')
                                                  ,axis = 1)

In [361]:
# Renombro columnas
df_bandera_21 = df_bandera_21.rename(columns = {'tipoprocesoseleccion_pac': 'bandera_21_tipoprocesoseleccion_pac',
                                               'tipoprocesoseleccion_adj': 'bandera_21_tipoprocesoseleccion_adj'})

In [362]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('21',start,end, tiempos)

### Bandera 22: El plazo transcurrido entre el consentimiento de buena pro y suscripción del contrato es demasiado largo



In [369]:
# Start time de bandera
start = datetime.datetime.now()

In [370]:
# Selecciono variables relevantes para la bandera
df_bandera22 = df_agg_cac[['codigoconvocatoria','anio','tipoprocesoseleccion']]

In [371]:
Bandera22_adj_contratos = adjudicaciones[['codigoconvocatoria','n_item','codigoitem','itemcubso','ruc_codigo_ganador','ganador','fecha_consentimiento_bp','tipo_proveedor','ruc_miembro']].join(contratos[['codigoconvocatoria','num_item','ruc_codigo_ganador','fecha_suscripcion_contrato']]\
                                              .set_index(['codigoconvocatoria','num_item','ruc_codigo_ganador']),
                                             on = ['codigoconvocatoria','n_item','ruc_codigo_ganador'],
                                             how = 'inner')

In [372]:
# Calculo diferencia de dias entre fechas
Bandera22_adj_contratos['fecha_consentimiento_bp'] = Bandera22_adj_contratos['fecha_consentimiento_bp'].fillna('1900-01-01') # nan a 1900 para poder computar np.busday
Bandera22_adj_contratos['fecha_suscripcion_contrato'] = Bandera22_adj_contratos['fecha_suscripcion_contrato'].fillna('1900-01-01') # nan a 1900 para poder computar np.busday

# Calculo diferencia de dias entre fechas excluyendo fin de semanas y feriados
Bandera22_adj_contratos['bandera_22_dif_consentimientobp_suscripcioncontrato'] = np.busday_count(Bandera22_adj_contratos['fecha_consentimiento_bp'].values.astype('datetime64[D]'),
                               Bandera22_adj_contratos['fecha_suscripcion_contrato'].values.astype('datetime64[D]'), 
                              holidays = fecha_feriados)

# Asigno NA a los casos en los que fecha convocatoria y/o fechapresentacionpropuesta es null
Bandera22_adj_contratos['bandera_22_dif_consentimientobp_suscripcioncontrato'] = Bandera22_adj_contratos.apply(lambda x: None if x['fecha_consentimiento_bp'] == '1900-01-01' or x['fecha_suscripcion_contrato'] == '1900-01-01'
                                                                                      else x['bandera_22_dif_consentimientobp_suscripcioncontrato'], axis = 1)

In [373]:
Bandera22_adj_contratos['consorcio'] = Bandera22_adj_contratos.apply(lambda x: x['ruc_codigo_ganador'] 
                                                                       if x['tipo_proveedor'] == 'Consorcio' else 'No',
                                                                      axis = 1)

Bandera22_adj_contratos['ruc_proveedor'] = Bandera22_adj_contratos.apply(lambda x: x['ruc_miembro'] 
                                                                       if x['tipo_proveedor'] == 'Consorcio' else x['ruc_codigo_ganador'],
                                                                      axis = 1)

In [374]:
# Calculo bandera a nivel item
umbral = 16
Bandera22_adj_contratos['bandera_22'] = Bandera22_adj_contratos.apply(lambda x: 'Si' if (x['bandera_22_dif_consentimientobp_suscripcioncontrato'] > umbral) and (x['bandera_22_dif_consentimientobp_suscripcioncontrato'] < 1000) else 'No', axis = 1)

In [375]:
# Me quedo solo con items para los que salto la bandera
Bandera22_adj_contratos = Bandera22_adj_contratos.loc[Bandera22_adj_contratos['bandera_22'] == 'Si']

In [376]:
# Cambio los tipos de datos de columnas que usare como atributos a string
Bandera22_adj_contratos['ruc_codigo_ganador'] = Bandera22_adj_contratos['ruc_codigo_ganador'].astype(str)
Bandera22_adj_contratos['ruc_proveedor'] = Bandera22_adj_contratos['ruc_proveedor'].astype(str)
Bandera22_adj_contratos['ganador'] = Bandera22_adj_contratos['ganador'].astype(str)
Bandera22_adj_contratos['consorcio'] = Bandera22_adj_contratos['consorcio'].astype(str)
Bandera22_adj_contratos['codigoitem'] = Bandera22_adj_contratos['codigoitem'].astype(str)
Bandera22_adj_contratos['itemcubso'] = Bandera22_adj_contratos['itemcubso'].astype(str)
Bandera22_adj_contratos['fecha_consentimiento_bp'] = Bandera22_adj_contratos['fecha_consentimiento_bp'].astype(str)
Bandera22_adj_contratos['fecha_suscripcion_contrato'] = Bandera22_adj_contratos['fecha_suscripcion_contrato'].astype(str)
Bandera22_adj_contratos['bandera_22_dif_consentimientobp_suscripcioncontrato'] = Bandera22_adj_contratos['bandera_22_dif_consentimientobp_suscripcioncontrato'].astype(str)

In [377]:
# Genero una columna que tendra todos los atributos que quiero - concatenando las columnas

Bandera22_adj_contratos['bandera22_atributos'] = 'Consorcio:' + Bandera22_adj_contratos['consorcio'] + ';'+\
' Ruc o Ruc miembro consorcio:' + Bandera22_adj_contratos['ruc_proveedor'] + ';'+\
' Razon social:' + Bandera22_adj_contratos['ganador'] + ';'+\
' CodigoItem:' + Bandera22_adj_contratos['codigoitem'] + ';'+ \
' Itemcubso:' + Bandera22_adj_contratos['itemcubso'] + ';'+ \
' Fecha consentimiento bp: ' + Bandera22_adj_contratos['fecha_consentimiento_bp'] + ';'+ \
' Fecha suscripcion contrato: ' + Bandera22_adj_contratos['fecha_suscripcion_contrato'] + ';'+ \
' Diferencia:' + Bandera22_adj_contratos['bandera_22_dif_consentimientobp_suscripcioncontrato']

In [378]:
# Flatten del df para que quede a nivel codigoconvocatoria y haga un string con todos los items con bandera22
Bandera22_adj_contratos = pd.DataFrame(Bandera22_adj_contratos.groupby(['codigoconvocatoria'], as_index = False)['bandera22_atributos'].apply(r'|'.join))



In [379]:
# Join df_bandera_22 con Bandera22_adj_contratos
df_bandera22 = df_bandera22.join(Bandera22_adj_contratos.set_index('codigoconvocatoria'),
                                on = 'codigoconvocatoria',
                                how = 'left')

In [380]:
# Cambio nan por None
df_bandera22=df_bandera22.where(pd.notnull(df_bandera22), None)

In [381]:
# Genero bandera 22 a nivel convocatoria - para todos excepto Contratacion Directa
df_bandera22['bandera_22'] = df_bandera22.apply(lambda x: 'No aplica' if x['tipoprocesoseleccion'] == 'Contratación Directa' 
                                               else ('No' if x['bandera22_atributos'] is None else 'Si'), axis = 1)



In [382]:
# Renombro atributos
df_bandera22 = df_bandera22.rename(columns = {'bandera22_atributos':'df_bandera22_itemscontratados_plazo_mayor_ley'})
# Agrego columna con umbral
df_bandera22['bandera_22_umbral'] = 16

In [383]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('22',start,end, tiempos)

### Bandera 23a

In [390]:
# Start time de bandera
start = datetime.datetime.now()

In [391]:
# Selecciono variables para construir bandera
df_bandera23a = df_agg_cac[['codigoconvocatoria','tipoprocesoseleccion','objetocontractual']]

In [392]:
# Selecciono variables de convocatorias para join con contratos
bandera_23a_convocatorias = convocatorias[['codigoconvocatoria','objetocontractual','tipoprocesoseleccion']].drop_duplicates()

In [393]:
# Selecciono variables para construir bandera - dataframe de contratos
bandera_23a_contratos = contratos[['codigoconvocatoria',
                                   'n_cod_contrato',
                                   'ruc_proveedor',
                                   'ruc_proveedor_razon_social',
                                   'monto_contratado_total',
                                   'monto_adicional',
                                  'monto_reduccion',
                                  'monto_complementario']]

# Join con convocatorias para traer proceso de seleccion y objeto contractual
bandera_23a_contratos = bandera_23a_contratos.join(bandera_23a_convocatorias.set_index('codigoconvocatoria'),
                                                  on = 'codigoconvocatoria',
                                                  how = 'inner')

In [394]:
# Flatten del dataframe a nivel cod-convocatoria-contrato
bandera_23a_contratos = bandera_23a_contratos.drop_duplicates(['codigoconvocatoria','n_cod_contrato'])

In [395]:
# Si los montos son Nan entonces convertir a cero
bandera_23a_contratos = bandera_23a_contratos.where(pd.notnull(bandera_23a_contratos),0)

In [396]:
bandera_23a_contratos['bandera_23a_adicional_vs_contratado'] = (bandera_23a_contratos['monto_adicional'] * 1.00 / bandera_23a_contratos['monto_contratado_total'] * 100)
bandera_23a_contratos['bandera_23a_reduccion_vs_contratado'] = bandera_23a_contratos['monto_reduccion'] * 1.00 / bandera_23a_contratos['monto_contratado_total'] * 100
bandera_23a_contratos['bandera_23a_complementario_vs_contratado'] = bandera_23a_contratos['monto_complementario'] * 1.00 / bandera_23a_contratos['monto_contratado_total'] * 100

In [397]:
# Si los montos son Nan entonces convertir a cero
bandera_23a_contratos = bandera_23a_contratos.where(pd.notnull(bandera_23a_contratos),0)

In [398]:
def generacion_microbanderas_23a(objetocontractual,tipoprocesoseleccion,bandera_23a_adicional_vs_contratado,bandera_23a_reduccion_vs_contratado,bandera_23a_complementario_vs_contratado):
    if objetocontractual in (['Bien','Servicio']):
        #### BIENES Y SERVICIOS ####
        # Adicional vs contratado
        if (bandera_23a_adicional_vs_contratado >= 23) & (bandera_23a_adicional_vs_contratado <= 25):
            resultado_adicional_vs_contratado = 'Si'
        else: 
            resultado_adicional_vs_contratado = 'No'
        # Reduccion vs contratado
        if (bandera_23a_reduccion_vs_contratado >= 23) & (bandera_23a_reduccion_vs_contratado <= 25):
            resultado_reduccion_vs_contratado = 'Si'
        else: 
            resultado_reduccion_vs_contratado = 'No'
        # Complementario vs contratado
        if tipoprocesoseleccion == 'Contratación Directa':
            resultado_complementario_vs_contratado = 'No aplica'
        elif (bandera_23a_complementario_vs_contratado >= 23) & (bandera_23a_complementario_vs_contratado <= 25):
            resultado_complementario_vs_contratado = 'Si'
        else: 
            resultado_complementario_vs_contratado = 'No'
    elif objetocontractual == 'Obra':
        ##### OBRAS #####
        # Adicional vs contratado
        if (bandera_23a_adicional_vs_contratado >= 47) & (bandera_23a_adicional_vs_contratado <= 50):
            resultado_adicional_vs_contratado = 'Si'
        else: 
            resultado_adicional_vs_contratado = 'No'
        # Reduccion vs contratado
        if (bandera_23a_reduccion_vs_contratado >= 47) & (bandera_23a_reduccion_vs_contratado <= 50):
            resultado_reduccion_vs_contratado = 'Si'
        else: 
            resultado_reduccion_vs_contratado = 'No'
        # Complementario vs contratado
        if (bandera_23a_complementario_vs_contratado >= 47) & (bandera_23a_complementario_vs_contratado <= 50):
            resultado_complementario_vs_contratado = 'Si'
        else: 
            resultado_complementario_vs_contratado = 'No'
    else:
        ##### CONSULTORIA DE OBRA ####
        resultado_adicional_vs_contratado = 'No aplica'
        resultado_reduccion_vs_contratado = 'No aplica'
        resultado_complementario_vs_contratado = 'No aplica'
        
    return(pd.Series([resultado_adicional_vs_contratado,resultado_reduccion_vs_contratado,resultado_complementario_vs_contratado]))
    

In [399]:
# Microbanderas a nivel contrato
bandera_23a_contratos[['bandera_23a_adicion','bandera_23a_reduccion','bandera_23a_complementario']] = bandera_23a_contratos.apply(lambda x: generacion_microbanderas_23a(x['objetocontractual'],
                                                                                                                                                                         x['tipoprocesoseleccion'],
                                                                                                                                                                         x['bandera_23a_adicional_vs_contratado'],
                                                                                                                                                                         x['bandera_23a_reduccion_vs_contratado'],
                                                                                                                                                                         x['bandera_23a_complementario_vs_contratado']),
                                                                  axis = 1)

In [400]:
# Bandera 23a a nivel contrato
bandera_23a_contratos['bandera_23a'] = bandera_23a_contratos.apply(lambda x: 'Si' if (x['bandera_23a_adicion'] == 'Si') 
                                                                   or (x['bandera_23a_reduccion'] == 'Si') 
                                                                   or (x['bandera_23a_complementario'] == 'Si')
                                                                   else 'No', axis = 1)

In [401]:
# Nan a 0
bandera_23a_contratos = bandera_23a_contratos.where(pd.notnull(bandera_23a_contratos),0)

In [402]:
bandera_23a_contratos['n_cod_contrato'] = round(bandera_23a_contratos['n_cod_contrato'].astype(float),0).astype('Int64')

In [403]:
# Generacion de atributos
def generar_atributos_bandera23a(n_cod_contrato,
                                 ruc_proveedor,
                                 ruc_proveedor_razon_social,
                                monto_contratado_total,
                                monto_adicional,
                                monto_reduccion,
                                monto_complementario,
                                bandera_23a_adicional_vs_contratado,
                                bandera_23a_reduccion_vs_contratado,
                                bandera_23a_complementario_vs_contratado,
                                bandera_23a_adicion,
                                bandera_23a_reduccion,
                                bandera_23a_complementario):
    
    # Resultado
    atributos_generales = ' N_Cod_Contrato:' + str(n_cod_contrato) + ';'\
                          ' Ruc proveedor:' + str(ruc_proveedor) + ';' \
                          ' Razon social:' + str(ruc_proveedor_razon_social) + ';'
    
    resultado = atributos_generales
    
    # Atributos bandera adicion
    atributos_adicion = '**Monto adicional vs contratado**' + ';' + \
                        ' Monto contratado: ' + str(round(monto_contratado_total,2)) + ';' +\
                        ' Monto adicional: ' + str(round(monto_adicional,2)) + ';' +\
                        ' Porcentaje: ' + str(round(bandera_23a_adicional_vs_contratado,2)) +  '%' 
    # Atributos bandera reduccion
    atributos_reduccion = '**Monto reduccion vs contratado**' + ';' + \
                            ' Monto contratado: ' + str(round(monto_contratado_total,2)) + ';' + \
                            ' Monto reduccion: ' + str(round(monto_reduccion,2)) + ';' + \
                            ' Porcentaje: ' + str(round(bandera_23a_reduccion_vs_contratado,2)) +  '%'
    # Atributos bandera complementario
    atributos_complementario = '**Monto complementario vs contratado**' +  ';' +\
                            ' Monto contratado: ' + str(round(monto_contratado_total,2)) + ';' + \
                            ' Monto complementario: ' + str(round(monto_complementario,2)) + ';' + \
                            ' Porcentaje: ' + str(round(bandera_23a_complementario_vs_contratado,2)) +  '%'
    if bandera_23a_adicion == 'Si':
        resultado = resultado + atributos_adicion
    if bandera_23a_reduccion == 'Si':
        resultado = resultado + atributos_reduccion
    if bandera_23a_complementario == 'Si':
        resultado = resultado + atributos_complementario
                
    return(resultado)

In [404]:
bandera_23a_contratos['bandera_23a_atributos'] = bandera_23a_contratos.apply(lambda x: generar_atributos_bandera23a(x['n_cod_contrato'],
                                                                                        x['ruc_proveedor'],
                                                                                        x['ruc_proveedor_razon_social'],
                                                                                        x['monto_contratado_total'],
                                                                                        x['monto_adicional'],
                                                                                        x['monto_reduccion'],
                                                                                        x['monto_complementario'],
                                                                                        x['bandera_23a_adicional_vs_contratado'],
                                                                                        x['bandera_23a_reduccion_vs_contratado'],
                                                                                        x['bandera_23a_complementario_vs_contratado'],
                                                                                        x['bandera_23a_adicion'],
                                                                                        x['bandera_23a_reduccion'],
                                                                                        x['bandera_23a_complementario']), axis = 1)


In [405]:
# Me quedo solo con contratos con bandera = si
bandera_23a_contratos = bandera_23a_contratos.loc[bandera_23a_contratos['bandera_23a'] == 'Si']

In [406]:
# Flatten a nivel contrato
bandera_23a_contratos = bandera_23a_contratos.groupby('codigoconvocatoria', as_index = False)['bandera_23a_atributos'].apply(' | '.join)

In [407]:
# Join con df bandera 23a
df_bandera23a = df_bandera23a.join(bandera_23a_contratos.set_index('codigoconvocatoria'),
                              on = 'codigoconvocatoria',
                              how = 'left')

In [408]:
# Nan reemplzo por None
df_bandera23a = df_bandera23a.where(pd.notnull(df_bandera23a),None)

In [409]:
# Genero Bandera a nivel CodConvocatoria
df_bandera23a['bandera_23a'] = df_bandera23a.apply(lambda x: 'No aplica' if x['objetocontractual'] == 'Consultoría de Obra' else ('No' if x['bandera_23a_atributos'] is None else 'Si'), axis = 1)

In [410]:
# Rename columnas
df_bandera23a = df_bandera23a.rename(columns = {'tipoprocesoseleccion': 'bandera_23a_tipoprocesoseleccion',
                                                'objetocontractual': 'bandera_23a_objetocontractual'})

In [411]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('23a',start,end, tiempos)

### Bandera 27: El monto con el cual el proveedor ganador postuló es mayor al monto contratado

In [416]:
# Start time de bandera
start = datetime.datetime.now()

In [417]:
df_bandera27 = df_agg_cac[['codigoconvocatoria']]

In [418]:
# Contratos
bandera_27_contratos = contratos[['codigoconvocatoria','n_cod_contrato','ruc_proveedor','ruc_proveedor_razon_social','num_item','monto_contratado_item']]

In [419]:
# Join contratos y convocatorias para traerme itemcubso
bandera_27_convocatorias = convocatorias[['codigoconvocatoria','n_item','itemcubso','codigoitem']].drop_duplicates()

bandera_27_contratos = bandera_27_contratos.join(bandera_27_convocatorias.set_index(['codigoconvocatoria','n_item']),
                                                on = ['codigoconvocatoria','num_item'],
                                                how = 'inner')

In [420]:
# Postores
bandera_27_postores = postores[['codigo_convocatoria','ruc_detallepostor','detalle_postor','n_item','monto']].drop_duplicates()

In [421]:
# Join contratos y postores
bandera_27_contratos_postores = bandera_27_contratos.join(bandera_27_postores.set_index(['codigo_convocatoria','ruc_detallepostor','n_item']),
                                                    on = ['codigoconvocatoria','ruc_proveedor','num_item'],
                                                    how = 'left')

In [422]:
# Si no pudo joinear entonces None
bandera_27_contratos_postores = bandera_27_contratos_postores.where(pd.notnull(bandera_27_contratos_postores),None)

In [423]:
# Diferencia entre monto postulado y monto contratado
bandera_27_contratos_postores['bandera_27_dif_montopostulacionitem_montocontratadoitem'] = bandera_27_contratos_postores['monto_contratado_item'] - bandera_27_contratos_postores['monto']

In [424]:
# bandera a nivel item
bandera_27_contratos_postores['bandera_27'] = bandera_27_contratos_postores.apply(lambda x: 'No aplica' if x['monto'] is None else ('Si' if x['bandera_27_dif_montopostulacionitem_montocontratadoitem'] > 0 else 'No'), axis = 1)

In [425]:
# Me quedo con items con bandera = 'si'
bandera_27_contratos_postores_si = bandera_27_contratos_postores.loc[bandera_27_contratos_postores['bandera_27'].isin(['Si'])]                                            
                                                                  

In [426]:
# Me quedo con items con bandera = 'no aplica'
bandera_27_contratos_postores_noaplica = bandera_27_contratos_postores.loc[bandera_27_contratos_postores['bandera_27'].isin(['No aplica'])]          

# Flatten a nivel convocatoria
bandera_27_contratos_postores_noaplica = bandera_27_contratos_postores_noaplica[['codigoconvocatoria']].drop_duplicates()
bandera_27_contratos_postores_noaplica['bandera_27_noaplica'] = 'No aplica'


In [427]:
bandera_27_contratos_postores_si['n_cod_contrato'] = bandera_27_contratos_postores_si['n_cod_contrato'].astype(float)

<ipython-input-427-31e1f1168354>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bandera_27_contratos_postores_si['n_cod_contrato'] = bandera_27_contratos_postores_si['n_cod_contrato'].astype(float)


In [428]:
bandera_27_contratos_postores_si['bandera_27_dif_montopostulacionitem_montocontratadoitem'] = round(bandera_27_contratos_postores_si['bandera_27_dif_montopostulacionitem_montocontratadoitem'].astype(float),2)
bandera_27_contratos_postores_si['n_cod_contrato'] = round(bandera_27_contratos_postores_si['n_cod_contrato'].astype(float),0).astype('Int64')
bandera_27_contratos_postores_si['monto_contratado_item'] = round(bandera_27_contratos_postores_si['monto_contratado_item'].astype(float),2)
bandera_27_contratos_postores_si['monto'] = round(bandera_27_contratos_postores_si['monto'].astype(float),2)

<ipython-input-428-2fb73220733f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bandera_27_contratos_postores_si['bandera_27_dif_montopostulacionitem_montocontratadoitem'] = round(bandera_27_contratos_postores_si['bandera_27_dif_montopostulacionitem_montocontratadoitem'].astype(float),2)
<ipython-input-428-2fb73220733f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bandera_27_contratos_postores_si['n_cod_contrato'] = round(bandera_27_contratos_postores_si['n_cod_contrato'].astype(float),0).astype('Int6

In [429]:
# Genero campo para atributos de bandera
                                                        
bandera_27_contratos_postores_si['bandera_27_atributos_si'] = 'N cod contrato: ' + bandera_27_contratos_postores_si['n_cod_contrato'].astype(str) + ';' +\
                                                        ' Ruc: ' + bandera_27_contratos_postores_si['ruc_proveedor'].astype(str) + ';' +\
                                                        ' Razon social: ' + bandera_27_contratos_postores_si['ruc_proveedor_razon_social'].astype(str) + ';' +\
                                                        ' Itemcubso: ' + bandera_27_contratos_postores_si['itemcubso'].astype(str) + ';' +\
                                                        ' Codigoitem: ' + bandera_27_contratos_postores_si['codigoitem'].astype(str) + ';' +\
                                                        ' MontoContratadoItem: ' + bandera_27_contratos_postores_si['monto_contratado_item'].astype(str)  + ';' +\
                                                        ' MontoPostulacionItem: ' + bandera_27_contratos_postores_si['monto'].astype(str)  + ';' +\
                                                        ' Diferencia: ' + bandera_27_contratos_postores_si['bandera_27_dif_montopostulacionitem_montocontratadoitem'].astype(str) 
                                                                                

<ipython-input-429-0639503d2e6a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bandera_27_contratos_postores_si['bandera_27_atributos_si'] = 'N cod contrato: ' + bandera_27_contratos_postores_si['n_cod_contrato'].astype(str) + ';' +\


In [430]:
# Convierto bandera_27_atributos a string para poder hacer apply y join
bandera_27_contratos_postores_si['bandera_27_atributos_si'] = bandera_27_contratos_postores_si['bandera_27_atributos_si'].where(pd.notnull(bandera_27_contratos_postores_si['bandera_27_atributos_si']), '')

<ipython-input-430-00e54eeabd13>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bandera_27_contratos_postores_si['bandera_27_atributos_si'] = bandera_27_contratos_postores_si['bandera_27_atributos_si'].where(pd.notnull(bandera_27_contratos_postores_si['bandera_27_atributos_si']), '')


In [431]:
# Flatten del atributo, para llevarlo a nivel codigo convocatoria
bandera_27_contratos_postores_si  = pd.DataFrame(bandera_27_contratos_postores_si.groupby(['codigoconvocatoria'], as_index = False)\
                                                     ['bandera_27_atributos_si'].apply(' | '.join))

In [432]:
# Join df_bandera27 con codconv que tienen al menos un item con bandera = 1
df_bandera27 = df_bandera27.join(bandera_27_contratos_postores_si.set_index('codigoconvocatoria'),
                                on = 'codigoconvocatoria',
                                how = 'left')
# Join df_bandera27 con df que a nivel convocatoria indica si 'no aplica'
df_bandera27 = df_bandera27.join(bandera_27_contratos_postores_noaplica.set_index('codigoconvocatoria'),
                                on = 'codigoconvocatoria',
                                how = 'left')

In [433]:
#Nan a None
df_bandera27 = df_bandera27.where(pd.notnull(df_bandera27),None)

In [434]:
# Genero atributos a nivel codigo convocatoria
df_bandera27['bandera_27_atributos'] = df_bandera27.apply(lambda x:  x['bandera_27_atributos_si'] 
                                                          if x['bandera_27_atributos_si'] is not None
                                                         else x['bandera_27_noaplica'], axis = 1)

In [435]:
# Bandera 27 a nivel codigo convocatoria
df_bandera27['bandera_27']= df_bandera27.apply(lambda x: 'No aplica' if x['bandera_27_atributos'] == 'No aplica' else 
                                               ('No' if x['bandera_27_atributos'] is None else 'Si'), axis = 1 )

In [436]:
# Borro columnas que ya no se usan
df_bandera27 = df_bandera27[['codigoconvocatoria','bandera_27_atributos','bandera_27']]

In [437]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('27',start,end, tiempos)

### Bandera 29: La diferencia entre el monto de referencia en convocatoria y monto contratado es excesiva


In [452]:
# Start time de bandera
start = datetime.datetime.now()

In [453]:
df_bandera29 = df_agg_cac[['codigoconvocatoria','objetocontractual']]

In [454]:
# Selecciono variables de tabla contratos para hacerlo a nivel item
bandera_29_contratos = contratos[['codigoconvocatoria','num_item','n_cod_contrato','ruc_proveedor','ruc_proveedor_razon_social','monto_referencial_item','monto_contratado_item']]

In [455]:
# Flatten de tabla convocatorias para joinear y traerme itemcubso
bandera_29_convocatorias = convocatorias[['codigoconvocatoria','n_item','itemcubso','codigoitem']].drop_duplicates()

In [456]:
bandera_29_contratos = bandera_29_contratos.join(bandera_29_convocatorias.set_index(['codigoconvocatoria','n_item']),
                                                on = ['codigoconvocatoria','num_item'],
                                                how = 'left')

In [457]:
# Nan a 0
bandera_29_contratos = bandera_29_contratos.where(pd.notnull(bandera_29_contratos),0)

In [458]:
# Calculo %
bandera_29_contratos['bandera_29_porcentaje_mreferenciaitem_mcontratoitem'] = ((bandera_29_contratos['monto_contratado_item'] * 1.00 - bandera_29_contratos['monto_referencial_item']) / bandera_29_contratos['monto_referencial_item'] * 1.00 ) * 100 



In [459]:
# Calculo bandera a nivel item
bandera_29_contratos['bandera_29'] = bandera_29_contratos.apply(lambda x: 'Si' if x['bandera_29_porcentaje_mreferenciaitem_mcontratoitem'] > 30 else 'No', axis = 1)

In [460]:
# Me quedo con los casos en los que bandera a nivel item = Si
bandera_29_contratos = bandera_29_contratos.loc[bandera_29_contratos['bandera_29'] == 'Si']

In [461]:
bandera_29_contratos['n_cod_contrato'] = bandera_29_contratos['n_cod_contrato'].astype('Int64')

In [462]:
# Genero una columna con atributos que me interesan
                                              
bandera_29_contratos['bandera_29_atributos'] = 'N cod contrato: ' + bandera_29_contratos['n_cod_contrato'].astype(str) + ';' +\
                                               ' Ruc proveedor: ' + bandera_29_contratos['ruc_proveedor'].astype(str) + ';' +\
                                               ' Razon social: ' + bandera_29_contratos['ruc_proveedor_razon_social'].astype(str) + ';' +\
                                               ' Itemcubso: ' + bandera_29_contratos['itemcubso'].astype(str) + ';' +\
                                               ' Codigoitem: ' + bandera_29_contratos['codigoitem'].astype(str) + ';' +\
                                               ' MontoRefItem: ' + bandera_29_contratos['monto_referencial_item'].astype(int).astype(str) + ';' +\
                                               ' MontoContratadoItem: ' + bandera_29_contratos['monto_contratado_item'].astype(int).astype(str) + ';' +\
                                               ' Porcentaje: ' + bandera_29_contratos['bandera_29_porcentaje_mreferenciaitem_mcontratoitem'].astype(int).astype(str) + '%'



In [463]:
# Agrupo a nivel codconvocatoria
bandera_29_contratos = pd.DataFrame(bandera_29_contratos.groupby(['codigoconvocatoria'],as_index = False)['bandera_29_atributos'].apply(' | '.join))

In [464]:
# Join con df_bandera29
df_bandera29 = df_bandera29.join(bandera_29_contratos.set_index('codigoconvocatoria'),
                                on = 'codigoconvocatoria',
                                how = 'left')

In [465]:
# None cuando no hizo join con bandera_29_contratos
df_bandera29 = df_bandera29.where(pd.notnull(df_bandera29),None)

In [466]:
# Genero bandera a nivel convocatoria
df_bandera29['bandera_29'] = df_bandera29.apply(lambda x: 'No aplica' if x['objetocontractual'] in (['Consultoría de Obra','Obra']) else ('No' if x['bandera_29_atributos'] is None else 'Si'), axis = 1)


In [467]:
# Rename de atributos
df_bandera29 = df_bandera29.rename(columns = {'bandera_29_atributos': 'bandera_29_contrato_item_montorefitem_vs_montocontratoitem'})

In [468]:
# End time de bandera
end = datetime.datetime.now()

# Calculo tiempo de procesamiento de bandera y agrego a DF
tiempos = registro_tiempo_procesamiento('29',start,end, tiempos)

## 5. Salidas

### Postprocesamiento: Vinculación de todas las banderas con dataframe con características de la convocatoria

In [475]:
# Selecciono informacion basica de la convocatoria
df = df_agg_cac[['anio','codigoconvocatoria','proceso','entidad','ruc_entidad','sector','tipoprocesoseleccion','objetocontractual','descripcion_proceso','fechaconvocatoria','fechapresentacionpropuesta','monto_convocatoria']]

In [476]:
df.shape

(143735, 12)

In [477]:
# Selecciono solo convocatorias 2021
df = df.loc[df['anio'] == '2021']

In [478]:
df.shape

(18363, 12)

In [479]:
# Agrego bandera 1
df_malla = df.join(df_bandera_1[['codigoconvocatoria','bandera_1_grupo_entidad','bandera_1_version','bandera_1_media_versionesPAC','bandera_1_std_versionesPAC','bandera_1_umbral_versionesPAC','bandera_1']].set_index('codigoconvocatoria'),
                  how = 'left',
                  on = 'codigoconvocatoria')

df_malla['bandera_1'].fillna('No aplica', inplace = True)



In [480]:
df_malla.shape

(18363, 18)

In [481]:
# Agrego bandera 2
df_malla = df_malla.join(df_bandera2[['codigoconvocatoria','bandera2_ente_%contrdirecta_item_mayor_2SD','bandera_2']].set_index('codigoconvocatoria'),
                  how = 'left',
                  on = 'codigoconvocatoria')


In [482]:
df_malla.shape

(18363, 20)

In [483]:
# Agrego bandera 3
df_malla = df_malla.join(df_bandera_3[['codigoconvocatoria','bandera_3_dif_convocatoria_presentacionpropuestas','bandera_3_umbral','bandera_3']].set_index('codigoconvocatoria'),
                  how = 'left',
                  on = 'codigoconvocatoria')

In [484]:
df_malla.shape

(18363, 23)

In [485]:
# Agrego bandera 3a
df_malla = df_malla.join(df_bandera_3a[['codigoconvocatoria','bandera_3a_dif_convocatoria_presentacionpropuestas','bandera_3a_media','bandera_3a_std','bandera_3a_umbral','bandera_3a']].set_index('codigoconvocatoria'),
                  how = 'left',
                  on = 'codigoconvocatoria')

In [486]:
df_malla.shape

(18363, 28)

In [487]:
# Agrego bandera 4
df_malla = df_malla.join(df_bandera_4[['codigoconvocatoria','bandera_4_cantidad_ruc_postores','bandera_4']].set_index('codigoconvocatoria'),
                  how = 'left',
                  on = 'codigoconvocatoria')

In [488]:
df_malla.shape

(18363, 30)

In [489]:
# Agrego bandera 4a
df_malla = df_malla.join(df_bandera_4a_2021[['codigoconvocatoria',
                                             'bandera_4a_cant_ruc_codigo_postor',
                                             'bandera_4a_cant_rucparticipante',
                                            'bandera_4a_ratio_postor_participante',
                                            'bandera_4a_media_ratio_postor_participante',
                                            'bandera_4a_std_ratio_postor_participante',
                                             'bandera_4a_umbral_ratio_postor_participante',
                                            'bandera_4a']].set_index('codigoconvocatoria'),
                  how = 'left',
                  on = 'codigoconvocatoria')

In [490]:
df_malla.shape

(18363, 37)

In [491]:
# Agrego bandera 5
df_malla = df_malla.join(df_bandera5[['codigoconvocatoria',
                                       'bandera_5_proveedores_primera_vez',
                                       'bandera_5']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [492]:
df_malla.shape

(18363, 39)

In [493]:
# Agrego bandera 5a
df_malla = df_malla.join(df_bandera5a_2021[['codigoconvocatoria',
                                       'bandera5a_items_nunca_antes_adjudicados_por_postor',
                                       'bandera_5a']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [494]:
df_malla.shape

(18363, 41)

In [495]:
# Agrego bandera 8
df_malla = df_malla.join(df_bandera8[['codigoconvocatoria',
                                       'bandera8_FechaAdj_Posterior_FechaDGR',
                                       'bandera_8']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [496]:
df_malla.shape

(18363, 43)

In [497]:
# Agrego bandera 13
df_malla = df_malla.join(df_bandera13[['codigoconvocatoria',
                                       'bandera_13_RUC_nohabido_nohallado',
                                       'bandera_13']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [498]:
df_malla.shape

(18363, 45)

In [499]:
# Agrego bandera 15
df_malla = df_malla.join(df_bandera15[['codigoconvocatoria',
                                       'bandera15_items_cercanos_valor_referencia',
                                       'bandera15']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [500]:
df_malla.shape

(18363, 47)

In [501]:
# Agrego bandera 16
df_malla = df_malla.join(df_bandera16[['codigoconvocatoria',
                                       'bandera_16_rucentidad','bandera_16_entidad',
                                        'bandera_16_postores_codflia_porcentaje_adj_vs_postulado',
                                       'bandera_16']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [502]:
df_malla.shape

(18363, 51)

In [503]:
# Agrego bandera 17
df_malla = df_malla.join(df_bandera_17[['codigoconvocatoria',
                                       'bandera_17_umbral','bandera17_items_adjudicados_plazo_inferior_a_ley',
                                       'bandera_17']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [504]:
df_malla.shape

(18363, 54)

In [505]:
# Agrego bandera 19
df_malla = df_malla.join(df_bandera19[['codigoconvocatoria',
                                       'bandera19_precio_unitario_item_vs_historico',
                                       'bandera19']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [506]:
df_malla.shape

(18363, 56)

In [507]:
# Agrego bandera 21
df_malla = df_malla.join(df_bandera_21[['codigoconvocatoria',
                                       'bandera_21_tipoprocesoseleccion_pac',
                                       'bandera_21_tipoprocesoseleccion_adj',
                                      'bandera_21']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [508]:
df_malla.shape

(18363, 59)

In [509]:
# Agrego bandera 22
df_malla = df_malla.join(df_bandera22[['codigoconvocatoria',
                                       'df_bandera22_itemscontratados_plazo_mayor_ley',
                                       'bandera_22_umbral',
                                      'bandera_22']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [510]:
df_malla.shape

(18363, 62)

In [511]:
# Agrego bandera 23a
df_malla = df_malla.join(df_bandera23a[['codigoconvocatoria',
                                       'bandera_23a_tipoprocesoseleccion',
                                       'bandera_23a_objetocontractual',
                                      'bandera_23a_atributos',
                                      'bandera_23a']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [512]:
df_malla.shape

(18363, 66)

In [513]:
# Agrego bandera 27
df_malla = df_malla.join(df_bandera27[['codigoconvocatoria',
                                       'bandera_27_atributos',
                                      'bandera_27']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [514]:
df_malla.shape

(18363, 68)

In [515]:
# Agrego bandera 29
df_malla = df_malla.join(df_bandera29[['codigoconvocatoria',
                                       'bandera_29_contrato_item_montorefitem_vs_montocontratoitem',
                                      'bandera_29']].set_index('codigoconvocatoria'),
                        how ='left',
                        on = 'codigoconvocatoria')

In [516]:
df_malla.shape

(18363, 70)

### Postprocesamiento: Etapas a la que pertenece la convocatoria

In [517]:
banderas = ['bandera_1',
            'bandera_2',
            'bandera_3',
            'bandera_3a',
            'bandera_4',
            'bandera_4a',
            'bandera_5',
            'bandera_5a',
            'bandera_8',
            'bandera_13',
            'bandera15',
            'bandera_16',
            'bandera_17',
            'bandera19',
            'bandera_21',
            'bandera_22',
           'bandera_23a',
           'bandera_27',
           'bandera_29']

In [518]:
etapa_convocatoria = ['bandera_3',
                     'bandera_3a',
                     'bandera_2']
etapa_adjudicacion = ['bandera_17',
                      'bandera_4',
                     'bandera_13',
                     'bandera19',
                     'bandera_8',
                     'bandera_21']
etapa_contrato = ['bandera_22',
                 'bandera_29',
                 'bandera_27']
etapa_presentacionpropuesta = ['bandera_5',
                               'bandera_4a',
                              'bandera_5a',
                              'bandera_16',
                              'bandera15']
etapa_ejecucion = ['bandera_23a']
etapa_planificacion = ['bandera_1']

In [519]:
for bandera in banderas:
    nombre_columna = f'{bandera}_etapa'
    if bandera in etapa_convocatoria:     
        df_malla[nombre_columna] = 'convocatoria'
    elif bandera in etapa_adjudicacion:
        df_malla[nombre_columna] = 'adjudicaciones'
    elif bandera in etapa_contrato:
        df_malla[nombre_columna] = 'contrato'
    elif bandera in etapa_presentacionpropuesta:
        df_malla[nombre_columna] = 'presentacion propuesta'
    elif bandera in etapa_ejecucion:
        df_malla[nombre_columna] = 'ejecucion'
    elif bandera in etapa_planificacion:
        df_malla[nombre_columna] = 'planificacion'

### Salida: Tiempo de procesamiento de cada bandera

In [523]:
tiempos['tiempo_procesamiento'] = tiempos['tiempo_procesamiento'].astype(str)

In [524]:
# Guardo tiempos a Excel
hoy = datetime.datetime.today().strftime('%d%b%Y')
tiempos.to_excel(f'banderas_rojas/banderas_rojas_tiempos_{hoy}.xlsx',index = False)

### Salida: Cantidad de alertas disparadas para cada bandera

In [525]:
def cuadro_resumen(df,variable,porcentaje = False):
    """
    Genera tabla que contabiliza la cantidad de ocurrencias de la variable dada. Con y sin porcentaje.
    
    Params
    -----
    df(dataframe): Dataframe que contiene la variable a agrupar y el codigodeconvocotaria
    variable(string): Variable a agrupar por ejemplo 'bandera_4'
    porcentaje (bool): Si es igual a  True agrega columna con porcentaje. 
    
    Returns
    -------
    Cuadro resumen con cantidad de ocurrencias para cada valor de la variable consultada.
    """
    objeto = df.groupby(variable, as_index = False).count()[[variable,'codigoconvocatoria']].rename(columns = {'codigoconvocatoria':'cantidad'}).sort_values(by = 'cantidad', ascending = True)
    if porcentaje:
        objeto['porcentaje'] = (objeto.cantidad*1.0 / objeto.cantidad.sum())*100
    objeto = objeto.sort_values(by = 'cantidad', ascending = False)
    total = objeto.sum()
    total.name = 'Total'
    total
    objeto = objeto.append(total.transpose())
    
    return(objeto)

In [1]:
# Cantidad de casos por bandera
for bandera in banderas:
    display(cuadro_resumen(df_malla,bandera,porcentaje = True))

In [530]:
# Funcion para cantidad de banderas disparadas
def contador_banderas_si(*args):
    contador = [1 for item in args if 'Si' in item]
    resultado = sum(contador)
    return(resultado)

In [531]:
# Cantidad de banderas disparadas
df_malla['cantidad_banderas_si'] = df_malla.apply(lambda x: contador_banderas_si(x['bandera_1'],
                                                                                x['bandera_2'],
                                                                                x['bandera_3'],
                                                                                x['bandera_3a'],
                                                                                x['bandera_4'],
                                                                                x['bandera_4a'],
                                                                                x['bandera_5'],
                                                                                x['bandera_5a'],
                                                                                x['bandera_8'],
                                                                                x['bandera_13'],
                                                                                x['bandera15'],
                                                                                x['bandera_16'],
                                                                                x['bandera_17'],
                                                                                x['bandera19'],
                                                                                x['bandera_21'],
                                                                                x['bandera_22'],
                                                                                x['bandera_23a'],
                                                                                x['bandera_27'],
                                                                                x['bandera_29']),
                                                  axis = 1)

In [532]:
# Genero cuadro para PPT
cuadro_ppt = pd.DataFrame()

In [533]:
# Genero cuadro para PPT
for bandera in banderas:
    resumen = cuadro_resumen(df_malla,bandera,porcentaje = True)
    resumen['bandera'] = bandera
    resumen_pivot = resumen.pivot(index='bandera', columns=str(bandera), values='cantidad')
    cuadro_ppt = cuadro_ppt.append(resumen_pivot)

In [535]:
cuadro_ppt = cuadro_ppt.fillna(0)
cuadro_ppt['Total'] = cuadro_ppt['NoNo aplicaSi'] + cuadro_ppt['NoNo aplica']+cuadro_ppt['No aplicaNoSi'] + cuadro_ppt['NoSi'] + cuadro_ppt['NoSiNo aplica']

In [537]:
cuadro_ppt = cuadro_ppt[['Total','Si']]

In [538]:
cuadro_ppt['%Si'] = (cuadro_ppt['Si'] * 1.00 / cuadro_ppt['Total'] * 1.00) * 100

<ipython-input-538-6e43d2afc1bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cuadro_ppt['%Si'] = (cuadro_ppt['Si'] * 1.00 / cuadro_ppt['Total'] * 1.00) * 100


In [539]:
cuadro_ppt.to_excel(f'banderas_rojas/banderas_rojas_cuadro_ppt_versionprevia_{hoy}.xlsx',index = True)

In [540]:
cuadro_ppt = cuadro_ppt.sort_values(by='%Si',ascending = True)

In [541]:
cuadro_ppt['%Si'] = cuadro_ppt['%Si'].map('{:,.2f}%'.format)

In [543]:
# Guardo cuadro en a Excel
hoy = datetime.datetime.today().strftime('%d%b%Y')
cuadro_ppt.to_excel(f'banderas_rojas/banderas_rojas_cuadro_ppt_{hoy}.xlsx',index = True)

### Salida: Cantidad de banderas disparadas por entidad

In [545]:
# Genero dataframe con todas las entidades
df_entidad = df_malla[['ruc_entidad','entidad']].drop_duplicates()

In [546]:
def cantidad_si_por_bandera_por_entidad(df_entidad, df_malla, bandera):
    df_ente_bandera = df_malla.loc[df_malla[bandera] == 'Si']\
    .groupby(['entidad',bandera], as_index = False).count()\
    [['entidad',bandera,'codigoconvocatoria']]\
    .rename(columns = {'codigoconvocatoria':'cantidad'})\
    .pivot('entidad',bandera,'cantidad').rename_axis(None, axis=1).reset_index()\
    .rename(columns = {'Si': bandera})
    
    df_entidad = df_entidad.join(df_ente_bandera.set_index('entidad'), on = 'entidad', how = 'left')
    
    return(df_entidad)
                       

In [547]:
for bandera in banderas:
    df_entidad = cantidad_si_por_bandera_por_entidad(df_entidad, df_malla, bandera)

In [548]:
# Reemplazo Nan por 0
df_entidad = df_entidad.fillna(0)

In [549]:
df_entidad['total'] = df_entidad.drop('ruc_entidad', axis=1).sum(axis=1)

In [550]:
#df_entidad = df_entidad.append(df_entidad.sum(numeric_only=True), ignore_index=True)

In [551]:
df_entidad = df_entidad[['ruc_entidad', 'entidad','total', 'bandera_1', 'bandera_2', 'bandera_3a',
       'bandera_4', 'bandera_4a', 'bandera_5', 'bandera_5a', 'bandera_8',
       'bandera_13', 'bandera15', 'bandera_16', 'bandera_17', 'bandera19',
       'bandera_21', 'bandera_22', 'bandera_23a', 'bandera_27', 'bandera_29'
       ]]

In [552]:
df_entidad = df_entidad.sort_values(by='total', ascending = False)

In [553]:
hoy = datetime.datetime.today().strftime('%d%b%Y')
df_entidad.to_excel(f'banderas_rojas/banderas_rojas_entidades_{hoy}.xlsx',index = False)

### Salida: Codigos y grupos

In [557]:
# grupos y codigos
adjudicaciones[['codigogrupo','descripcion_grupo']].drop_duplicates().to_excel(f'banderas_rojas/banderas_rojas_codigosgrupos_gruposdescripcion.xlsx',index = False)

### Salida: Malla en Excel

In [555]:
# Quito new lines de descpcion de proceso
df_malla['descripcion_proceso'] = df_malla['descripcion_proceso'].replace(r'\s+|\\n', ' ', regex=True)

In [556]:
# Guardo malla a Excel
hoy = datetime.datetime.today().strftime('%d%b%Y')
df_malla.to_excel(f'banderas_rojas/banderas_rojas_malla_{hoy}.xlsx',index = False)